In [1]:
# %pip install -r requirements.txt --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --user

# Alternativa 01: Montar dados de publicações pelo Qlattes
    Passo01: Consultar cada nome, atribuir a àrea de avaliação e baixar o arquivo .csv para pasta _data/in_csv/qlattes
    Passo02: Rodar os scripts para concatenar os vários .csv com dados de publicações e gerar os dados de publicação

In [2]:
import pandas as pd
import os
from pathlib import Path

class AgregadorQlattes:
    def __init__(self):
        self.pasta_dados = 'ppgcs/_data/powerbi'
        self.arquivo_qualis_veiculos = 'MEDICINA_II1_(alterado_cristiana_2022-02-21).xlsx'
        self.pathfilename = Path.home() / self.pasta_dados / self.arquivo_qualis_veiculos
        self.nome_aba_veiculos = 'Veículos - Área'
        self.df_veiculos_qualis = self.ler_aba_excel(self.pathfilename, self.nome_aba_veiculos)
    
    @staticmethod
    def concatenar_csvs(diretorio):
        dfs = []
        for arquivo in os.listdir(diretorio):
            if arquivo.endswith('.csv'):
                caminho_completo = os.path.join(diretorio, arquivo)
                dfs.append(pd.read_csv(caminho_completo))
        df_concat = pd.concat(dfs, ignore_index=True)
        return df_concat

    @staticmethod
    def contar_publicacoes_periodo(df_concat, ano_inicio, ano_final):
        """
        Conta o número de publicações por qualis dentro de um intervalo de anos especificado.

        Parâmetros:
        df_concat (pandas.DataFrame): DataFrame contendo as publicações.
        ano_inicio (int): Ano de início do intervalo para considerar na contagem.
        ano_final (int): Ano de término do intervalo para considerar na contagem.

        Retorna:
        pandas.DataFrame: DataFrame contendo a contagem de publicações por qualis.
        """
        # Filtra o DataFrame com base no intervalo de anos especificado
        df_filtrado = df_concat[(df_concat['ano_publicacao'] >= ano_inicio) & (df_concat['ano_publicacao'] <= ano_final)]

        # Conta as publicações por qualis no DataFrame filtrado
        contagem_publicacoes_periodo = df_filtrado.groupby(['qualis']).size().reset_index(name='contagem')
        
        return contagem_publicacoes_periodo
    
    @staticmethod
    def contar_publicacoes_por_ano(df_concat, ano_inicio, ano_final):
        """
        Conta o número de publicações por qualis para cada ano dentro de um intervalo de anos especificado,
        adicionando ao final uma coluna com a quantidade total do período.

        Parâmetros:
        df_concat (pandas.DataFrame): DataFrame contendo as publicações.
        ano_inicio (int): Ano de início do intervalo para considerar na contagem.
        ano_final (int): Ano de término do intervalo para considerar na contagem.

        Retorna:
        pandas.DataFrame: DataFrame contendo a contagem de publicações por qualis para cada ano, e a soma total para o período.
        """
        # Filtra o DataFrame com base no intervalo de anos especificado
        df_filtrado = df_concat[(df_concat['ano_publicacao'] >= ano_inicio) & (df_concat['ano_publicacao'] <= ano_final)]

        # Conta as publicações por qualis e ano
        contagem = df_filtrado.groupby(['qualis', 'ano_publicacao']).size().reset_index(name='contagem')
        
        # Pivotagem para transformar os anos em colunas e qualis em linhas
        df_pivot = contagem.pivot_table(index='qualis', columns='ano_publicacao', values='contagem', fill_value=0)

        # Adiciona a coluna de soma total para o período
        df_pivot['Quantidade Total'] = df_pivot.sum(axis=1)
        
        return df_pivot.reset_index()

    @staticmethod
    def contar_publicacoes_nome(df_concat):
        contagem_publicacoes = df_concat.groupby(['nome','qualis']).size().reset_index(name='contagem')
        return contagem_publicacoes

    @staticmethod
    def contar_publicacoes_por_nome_ano(df_concat, ano_inicio, ano_final):
        """
        Conta o número de publicações por qualis para cada ano dentro de um intervalo de anos especificado,
        adicionando ao final uma coluna com a quantidade total do período.

        Parâmetros:
        df_concat (pandas.DataFrame): DataFrame contendo as publicações.
        ano_inicio (int): Ano de início do intervalo para considerar na contagem.
        ano_final (int): Ano de término do intervalo para considerar na contagem.

        Retorna:
        pandas.DataFrame: DataFrame contendo a contagem de publicações por qualis e por nome para cada ano, e a soma total para o período.
        """
        # Filtra o DataFrame com base no intervalo de anos especificado
        df_filtrado = df_concat[(df_concat['ano_publicacao'] >= ano_inicio) & (df_concat['ano_publicacao'] <= ano_final)]

        # Conta as publicações por qualis e ano
        contagem = df_filtrado.groupby(['qualis','nome','ano_publicacao']).size().reset_index(name='contagem')
        
        # Pivotagem para transformar os anos em colunas e qualis em linhas
        df_pivot = contagem.pivot_table(index='qualis', columns='ano_publicacao', values='contagem', fill_value=0)

        # Adiciona a coluna de soma total para o período
        df_pivot['Quantidade Total'] = df_pivot.sum(axis=1)
        
        return df_pivot.reset_index()

    @staticmethod
    def contar_titulos_publicacoes(df_concat):
        contagem_titulos_publicacoes = df_concat.groupby(['titulo_publicacao']).size().reset_index(name='contagem')
        return contagem_titulos_publicacoes

    @staticmethod
    def contar_qualis_ano(df, ano_inicio, ano_final):
        df_filtrado = df[(df['ano_publicacao'] >= ano_inicio) & (df['ano_publicacao'] <= ano_final)]
        tabela_pivot = df_filtrado.pivot_table(
            index='qualis',
            columns='ano_publicacao',
            values='titulo_publicacao',
            aggfunc='count'
        ).fillna(0)
        tabela_pivot['Total'] = tabela_pivot.sum(axis=1)
        return tabela_pivot

    @staticmethod
    def contar_qualis_nome_ano(df, ano_inicio, ano_final):
        df_filtrado = df[(df['ano_publicacao'] >= ano_inicio) & (df['ano_publicacao'] <= ano_final)]
        tabela_pivot = df_filtrado.pivot_table(
            index=['qualis','nome'],
            columns='ano_publicacao',
            values='titulo_publicacao',
            aggfunc='count'
        ).fillna(0)
        tabela_pivot['Total'] = tabela_pivot.sum(axis=1)
        return tabela_pivot


    @staticmethod
    def somar_pontuacao_por_nome_ano(df, qualis_pontos, ano_inicio, ano_final):
        """
        Calcula a soma das pontuações para cada nome em cada ano especificado, 
        apresentando a lista de nomes na vertical e a lista de anos com soma de pontuação como colunas.

        Parâmetros:
        df (pandas.DataFrame): DataFrame contendo as publicações.
        qualis_pontos (dict): Dicionário com a pontuação de cada qualis.
        ano_inicio (int): Ano de início do intervalo para considerar na contagem.
        ano_final (int): Ano de término do intervalo para considerar na contagem.

        Retorna:
        pandas.DataFrame: DataFrame com nomes na vertical e anos com soma de pontuação como colunas.
        """
        # Cria uma cópia do DataFrame filtrado para evitar SettingWithCopyWarning
        df_filtrado = df[(df['ano_publicacao'] >= ano_inicio) & (df['ano_publicacao'] <= ano_final)].copy()

        # Calcula a pontuação com base no qualis
        df_filtrado['pontuacao_calculada'] = df_filtrado['qualis'].map(qualis_pontos).fillna(0)

        # Agrupa por 'nome' e 'ano_publicacao', somando os pontos calculados
        df_agrupado = df_filtrado.groupby(['nome', 'ano_publicacao'])['pontuacao_calculada'].sum().reset_index()

        # Pivotagem para transformar os anos em colunas, nomes em linhas, e somar pontuações
        pontuacao_por_nome_ano = df_agrupado.pivot_table(index='nome', columns='ano_publicacao', values='pontuacao_calculada', aggfunc='sum', fill_value=0)
        pontuacao_por_nome_ano['Pontos no Período'] = pontuacao_por_nome_ano.sum(axis=1)

        return pontuacao_por_nome_ano

    @staticmethod
    def listar_nomes_abas(caminho_arquivo):
        xl = pd.ExcelFile(caminho_arquivo)
        nomes_abas = xl.sheet_names
        return nomes_abas

    @staticmethod
    def ler_aba_excel(caminho_arquivo, nome_aba):
        df = pd.read_excel(caminho_arquivo, sheet_name=nome_aba)
        return df


## Consolidar dados dos arquivos CSV do Q-Lattes

In [4]:
import os
qlattes_folder = './_data/in_csv/qlattes'
csvs = os.listdir(qlattes_folder)
print(f'{len(csvs)} arquivos de dados de currículos')
for i in csvs:
    print(i)
# pd.read_csv('.')

54 arquivos de dados de currículos
alessandra_aparecida_guarneri.csv
alexandre_de_magalhaes_vieira_machado.csv
alvaro_gil_araujo_ferreira.csv
ana_lucia_teles_rabello.csv
andrea_teixeira_de_carvalho.csv
antonio_mauro_rezende.csv
carina_margonari_de_souza.csv
carlos_eduardo_calzavara_silva.csv
caroline_furtado_junqueira.csv
celia_maria_ferreira_gontijo.csv
cristiana_couto_garcia.csv
cristiana_ferreira_alves_de_brito.csv
cristina_toscano_fonseca.csv
daniel_moreira_de_avelar.csv
edelberto_santos_dias.csv
edward_jose_de_oliveira.csv
erica_alessandra_rocha_alves.csv
erika_michalsky_monteiro.csv
flora_satiko_kano.csv
gabriel_da_rocha_fernandes.csv
glaucia_fernandes_cota.csv
gustavo_fontes_paz.csv
jaquelline_germano_de_oliveira.csv
jeronimo_conceicao_ruiz.csv
jose_dilermando_andrade_filho.csv
lileia_goncalves_diotaiuti.csv
lis_ribeiro_do_valle_antonelli.csv
luciano_andrade_moreira.csv
luiz_carlos_junior_alcantara.csv
luzia_helena_carvalho.csv
marcelo_antonio_pascoal_xavier.csv
marcio_sobreira_

In [5]:
## Agregar arquivos .csv de contagem extraídos com qlattes e atibuir pontuação por qualis
agregador = AgregadorQlattes()

qualis_pontos = {'A1': 90,'A2': 80,'A3': 60,'A4': 40,'B1': 20,'B2': 15,'B4': 5,'C': 0,'N':0}
df_publicacao_ppgcs = agregador.concatenar_csvs(qlattes_folder)
df_publicacao_ppgcs['pontos']=df_publicacao_ppgcs['pontos'] = df_publicacao_ppgcs['qualis'].map(qualis_pontos)
pd.set_option('display.max_rows', 600)
df_publicacao_ppgcs[1000:1500]

,nome,lattes_url,ano_publicacao,titulo_publicacao,periodico,issn,qualis,pontos,area,ano_base
1000,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2017,Seroprevalence and molecular characterization ...,INTERNATIONAL JOURNAL OF VETERINARY SCIENCE AN...,2314-4599,B1,20.0,NaN,2020.0
1001,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2015,A new species of perforating the osteoderms of...,MEDICAL AND VETERINARY ENTOMOLOGY (PRINT),0269-283X,A1,90.0,NaN,2020.0
1002,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2015,Phlebotomine Sand Fly Fauna and Infection in t...,BIOMED RES INT,2314-6133,A3,60.0,NaN,2020.0
1003,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2015,Dynamics of Ctenocephalides felis felis (Sipho...,JOURNAL OF MEDICAL ENTOMOLOGY,0022-2585,A1,90.0,NaN,2020.0
1004,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2015,"Epidemiological aspects of vector, parasite, a...",ACTA TROPICA,0001-706X,A2,80.0,NaN,2020.0
1005,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2015,"Leishmania, Babesia and Ehrlichia in urban pet...",REVISTA DA SOCIEDADE BRASILEIRA DE MEDICINA TR...,0037-8682,B1,20.0,NaN,2020.0
1006,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2014,Infection with Leishmania (Leishmania) infantu...,THE AMERICAN JOURNAL OF TROPICAL MEDICINE AND ...,0002-9637,A3,60.0,NaN,2020.0
1007,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2014,"Antibody responses induced by Leish-Tec®, an A...",VETERINARY PARASITOLOGY (PRINT),0304-4017,A1,90.0,NaN,2020.0
1008,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2014,Neosomes of tungid fleas on wild and domestic ...,PARASITOLOGY RESEARCH (1987. PRINT),0932-0113,A1,90.0,NaN,2020.0
1009,Daniel Moreira de Avelar,http://lattes.cnpq.br/3544791535477213,2014,Notes on the genus (Siphonaptera: Tungidae) II...,PARASITE,1776-1042,A3,60.0,NaN,2020.0


In [6]:
df_publicacao_ppgcs.replace('\r\n\t', '', regex=True, inplace=True)

In [7]:
# Salvar o DataFrame como arquivo CSV, separado por ';'
csv_filename = 'publicacoes_qlattes.csv'
filepath = os.path.join('_data','powerbi',csv_filename)
df_publicacao_ppgcs.to_csv(filepath, sep=';', index=False)

In [8]:
df_publicacao_ppgcs['qualis'].value_counts()

qualis
A1    1553
A2    1102
A4     783
A3     589
B1     409
N      273
B2     152
C       41
B3      35
B4      32
Name: count, dtype: int64

In [9]:
df_publicacao_ppgcs['pontos'].value_counts()

pontos
90.0    1553
80.0    1102
40.0     783
60.0     589
20.0     409
0.0      314
15.0     152
5.0       32
Name: count, dtype: int64

## Pontuar publicações no período selecionado (dados QLattes)

In [10]:
agregador = AgregadorQlattes()

ano_inicio=2017
ano_final=2024

In [11]:
qte_pub_por_qualis_ano = agregador.contar_qualis_ano(df_publicacao_ppgcs, ano_inicio, ano_final)
qte_pub_por_qualis_ano

ano_publicacao,2017,2018,2019,2020,2021,2022,2023,2024,Total
qualis,,,,,,,,,
A1,100.0,102.0,87.0,123.0,90.0,75.0,77.0,14.0,668.0
A2,69.0,64.0,64.0,82.0,94.0,125.0,71.0,8.0,577.0
A3,38.0,19.0,27.0,13.0,32.0,34.0,39.0,1.0,203.0
A4,25.0,31.0,36.0,43.0,11.0,22.0,28.0,4.0,200.0
B1,15.0,16.0,21.0,20.0,11.0,16.0,13.0,1.0,113.0
B2,11.0,12.0,3.0,8.0,8.0,5.0,3.0,2.0,52.0
B3,0.0,3.0,0.0,2.0,3.0,0.0,1.0,0.0,9.0
B4,2.0,5.0,1.0,0.0,3.0,1.0,2.0,0.0,14.0
C,2.0,4.0,5.0,2.0,5.0,8.0,2.0,2.0,30.0


In [12]:
pontos_por_nome_ano = agregador.somar_pontuacao_por_nome_ano(df_publicacao_ppgcs,qualis_pontos, ano_inicio, ano_final)
print(f'{len(pontos_por_nome_ano.index)}  Currículos sumarizados')
pontos_por_nome_ano.sort_values(by='Pontos no Período', ascending=False)

54  Currículos sumarizados


ano_publicacao,2017,2018,2019,2020,2021,2022,2023,2024,Pontos no Período
nome,,,,,,,,,
Olindo Assis Martins Filho,1505,1120,1850,1720,1475,1850,1465,170,11155
Luiz Carlos Júnior Alcantara,430,650,885,1300,1560,2025,1610,435,8895
Andréa Teixeira de Carvalho,1135,1115,1370,1555,720,1610,930,250,8685
Ricardo Tostes Gazzinelli,600,620,430,790,940,790,500,0,4670
Gabriel da Rocha Fernandes,920,370,350,540,150,825,765,150,4070
Glaucia Fernandes Cota,570,545,310,620,610,430,290,360,3735
Rodrigo Correa de Oliveira,1290,360,770,360,360,230,270,0,3640
Lis Ribeiro do Valle Antonelli,310,630,690,470,670,530,270,0,3570
Rodrigo Pedro Pinto Soares,540,560,620,520,750,150,280,130,3550


In [13]:
data_folder = '_data/in_csv/ppgcs'
os.listdir(data_folder)

['docentes_dadosprograma.csv',
 'docentes_nomes.csv',
 'docentes_todos.csv',
 'indicadores_quadrienio_17-20.csv']

## Ler a pasta de dados e remover acentuação dos nomes

In [15]:
import numpy as np
from unicodedata import normalize

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

filename = 'docentes_todos.csv'
filepath = os.path.join(data_folder,filename)

# Definir nomes das colunas
nomes_das_colunas = ['nome', 'id_lattes', 'programa','papel_programa']

# Ler arquivo CSV e nomear colunas especificados
df_nomes = pd.read_csv(filepath, names=nomes_das_colunas, header=None)
lista_nomes_acentuada = df_nomes['nome']

# lista_nomes.values sendo array do qual queremos remover acentos
lista_nomes = lista_nomes_acentuada.values

# Aplicar função remover_acentos a cada elemento do array da lista de nomes a extrair
lista_nomes_sem_acentos = np.vectorize(remover_acentos)(lista_nomes)

# Aplicar função remover_acentos a cada elemento extraído do QLattes
nomes_extraidos_sem_acentos = np.vectorize(remover_acentos)(pontos_por_nome_ano.index)

In [17]:
# Exemplo de uso da função para remover acentos
texto_corrigido = remover_acentos('Jerônimo Conceição Ruiz')
print(texto_corrigido)

Jeronimo Conceicao Ruiz


## Verificar se dados de todos currículos foram baixados

In [16]:
print(f'{len(lista_nomes_sem_acentos)} Nomes a extrair')
for i in lista_nomes_sem_acentos:
    if i not in nomes_extraidos_sem_acentos:
        print(f'Falta extrair no Qlattes: {i}')

55 Nomes a extrair
Falta extrair no Qlattes: Marcelo Gustavo Lorenzo


## Tabular dados concatenados

In [20]:
qte_pub_por_qualis_ano = agregador.contar_qualis_ano(df_publicacao_ppgcs, ano_inicio, ano_final)
qte_pub_por_qualis_ano

ano_publicacao,2017,2018,2019,2020,2021,2022,2023,2024,Total
qualis,,,,,,,,,
A1,100.0,102.0,87.0,123.0,90.0,75.0,77.0,14.0,668.0
A2,69.0,64.0,64.0,82.0,94.0,125.0,71.0,8.0,577.0
A3,38.0,19.0,27.0,13.0,32.0,34.0,39.0,1.0,203.0
A4,25.0,31.0,36.0,43.0,11.0,22.0,28.0,4.0,200.0
B1,15.0,16.0,21.0,20.0,11.0,16.0,13.0,1.0,113.0
B2,11.0,12.0,3.0,8.0,8.0,5.0,3.0,2.0,52.0
B3,0.0,3.0,0.0,2.0,3.0,0.0,1.0,0.0,9.0
B4,2.0,5.0,1.0,0.0,3.0,1.0,2.0,0.0,14.0
C,2.0,4.0,5.0,2.0,5.0,8.0,2.0,2.0,30.0


In [21]:
pd.set_option('display.max_rows', 600)
qte_pub_qualis_nome_ano = agregador.contar_qualis_nome_ano(df_publicacao_ppgcs, ano_inicio, ano_final)
qte_pub_qualis_nome_ano

ano_publicacao                                2017  2018  2019  2020  2021  \
qualis nome                                                                  
A1     Alessandra Aparecida Guarneri           3.0   2.0   1.0   3.0   2.0   
       Alexandre de Magalhaes Vieira Machado   0.0   0.0   1.0   1.0   1.0   
       Ana Lúcia Teles Rabello                 5.0   1.0   5.0   2.0   0.0   
       Andréa Teixeira de Carvalho             5.0   6.0   5.0   8.0   2.0   
       Antonio Mauro Rezende                   1.0   2.0   2.0   2.0   1.0   
       Carina Margonari de Souza               0.0   1.0   0.0   0.0   3.0   
       Carlos Eduardo Calzavara Silva          0.0   1.0   0.0   1.0   1.0   
       Caroline Furtado Junqueira              2.0   1.0   1.0   4.0   2.0   
       Celia Maria Ferreira Gontijo            4.0   3.0   1.0   2.0   2.0   
       Cristiana Couto Garcia                  1.0   0.0   0.0   2.0   1.0   
       Cristiana Ferreira Alves de Brito       1.0   4.0   3.0   3.0   1.0   
       Cristina Toscano Fonseca                1.0   1.0   0.0   0.0   0.0   
       Daniel Moreira de Avelar                0.0   0.0   1.0   0.0   1.0   
       Edelberto Santos Dias                   0.0   1.0   2.0   1.0   0.0   
       Edward José de Oliveira                 1.0   0.0   2.0   2.0   1.0   
       Erika Michalsky Monteiro                0.0   1.0   2.0   1.0   0.0   
       Flora Satiko Kano                       2.0   3.0   1.0   2.0   1.0   
       Gabriel da Rocha Fernandes              8.0   3.0   3.0   4.0   1.0   
       Glaucia Fernandes Cota                  3.0   5.0   3.0   4.0   3.0   
       Gustavo Fontes Paz                      4.0   1.0   1.0   2.0   2.0   
       Jaquelline Germano de Oliveira          0.0   0.0   0.0   1.0   0.0   
       Jeronimo Conceição Ruiz                 3.0   0.0   0.0   3.0   1.0   
       José Dilermando Andrade Filho           7.0   1.0   1.0   4.0   3.0   
       Lileia Gonçalves Diotaiuti              2.0   0.0   0.0   1.0   2.0   
       Lis Ribeiro do Valle Antonelli          1.0   5.0   5.0   3.0   5.0   
       Luciano Andrade Moreira                 4.0   3.0   3.0   4.0   4.0   
       Luiz Carlos Júnior Alcantara            3.0   3.0   7.0   8.0  10.0   
       Luzia Helena Carvalho                   1.0   3.0   1.0   4.0   1.0   
       Marcelo Antonio Pascoal Xavier          1.0   3.0   0.0   3.0   2.0   
       Marco Antônio da Silva Campos           0.0   1.0   0.0   0.0   0.0   
       Marina de Moraes Mourão                 2.0   1.0   1.0   1.0   1.0   
       Márcio Sobreira Silva Araújo            0.0   3.0   2.0   0.0   1.0   
       Nagila Francinete Costa Secundino       0.0   4.0   2.0   5.0   3.0   
       Olindo Assis Martins Filho              5.0   5.0   7.0   7.0   4.0   
       Paloma Helena Fernandes Shimabukuro     1.0   1.0   0.0   3.0   1.0   
       Paulo Filemon Paolucci Pimenta          0.0   7.0   3.0   6.0   2.0   
       Paulo Marcos Zech Coelho                3.0   1.0   2.0   2.0   0.0   
       Pedro Augusto Alves                     0.0   1.0   1.0   4.0   0.0   
       Rafaella Fortini Grenfell e Queiroz     0.0   0.0   2.0   1.0   2.0   
       Ricardo Tostes Gazzinelli               4.0   6.0   3.0   7.0   8.0   
       Rita de Cássia Moreira de Souza         3.0   0.0   2.0   1.0   2.0   
       Roberta Lima Caldeira                   3.0   0.0   0.0   0.0   1.0   
       Rodrigo Correa de Oliveira              7.0   2.0   5.0   2.0   2.0   
       Rodrigo Pedro Pinto Soares              4.0   2.0   2.0   0.0   4.0   
       Roney Santos Coimbra                    4.0   3.0   0.0   1.0   0.0   
       Rosiane Aparecida da Silva Pereira      0.0   1.0   1.0   0.0   0.0   
       Rubens Lima do Monte Neto               0.0   2.0   0.0   2.0   2.0   
       Silvane Maria Fonseca Murta             0.0   2.0   0.0   0.0   0.0   
       Soraya Torres Gaze Jangola              0.0   0.0   0.0   0.0   1.0   
       Taís Nóbrega de

In [22]:
## Avaliar multiplicidade de títulos
agregador.contar_titulos_publicacoes(df_publicacao_ppgcs).sort_values(by='contagem', ascending=False)

,titulo_publicacao,contagem
3217,Serum soluble mediators as prognostic biomarke...,8
1062,Distinct patterns of cellular immune response ...,7
2471,Multi-parameter approach to evaluate the timin...,6
3209,Serotype-associated immune response and networ...,6
3210,Serum Soluble Mediator Profiles and Networks D...,6
...,...,...
1757,IL-10 Limits Parasite Burden and Protects agai...,1
1758,IL-10 and TGF-β unbalanced levels in neutrophi...,1
1759,IL-10 inhibits parasite killing and nitrogen o...,1
1760,IL-10 produced by CD4(+) and CD8(+) T cells em...,1


In [23]:
import pandas as pd

def filtrar_por_periodo(df, ano_inicio, ano_final):
    """
    Filtra um DataFrame baseado em um intervalo de anos de publicação.

    Parâmetros:
    df (pandas.DataFrame): DataFrame contendo as publicações.
    ano_inicio (int): Ano de início do intervalo de filtro.
    ano_final (int): Ano de término do intervalo de filtro.

    Retorna:
    pandas.DataFrame: Novo DataFrame contendo apenas as publicações dentro do intervalo especificado.
    """
    # Filtra o DataFrame baseado nos anos de início e término fornecidos
    df_filtrado = df[(df['ano_publicacao'] >= ano_inicio) & (df['ano_publicacao'] <= ano_final)]
    
    return df_filtrado

# Exemplo de uso
ano_inicio=2017
ano_final=2020
dados_2017_2020 = filtrar_por_periodo(df_publicacao_ppgcs, ano_inicio, ano_final)
qte_pub_por_qualis_2017_2020 = agregador.contar_publicacoes_nome(dados_2017_2020)
qte_pub_por_qualis_2017_2020

,nome,qualis,contagem
0,Alessandra Aparecida Guarneri,A1,9
1,Alessandra Aparecida Guarneri,A2,4
2,Alessandra Aparecida Guarneri,A4,2
3,Alexandre de Magalhaes Vieira Machado,A1,2
4,Alexandre de Magalhaes Vieira Machado,A2,5
5,Alexandre de Magalhaes Vieira Machado,A3,2
6,Alexandre de Magalhaes Vieira Machado,B4,1
7,Ana Lúcia Teles Rabello,A1,13
8,Ana Lúcia Teles Rabello,A2,3
9,Ana Lúcia Teles Rabello,A3,1


# Alternativa 02: Usar dados do eLattes
    Passo01: Salvar o arquivo geral zipado extraído do e-lattes na pasta _data/in_json
    Passo02: Rodar scripts para gerar JSON único com dados de publicações e salvar em CSV


In [ ]:
import os
import re
import csv
import json
import time
import json
import zipfile
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from string import Formatter
from datetime import date, timedelta
from datetime import datetime as dt
from unidecode import unidecode
from plotly.subplots import make_subplots
from pyjarowinkler.distance import get_jaro_distance
from IPython.display import clear_output, display, HTML

pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')

class PreparadorDePublicacoes:
    def __init__(self):
        self.data = []
        self.colunas = ['idLattes', 'nome', 'tipo', 'titulo_do_capitulo', 'idioma', 'titulo_do_livro', 'ano', 'doi', 'pais_de_publicacao', 'isbn', 'nome_da_editora', 'numero_da_edicao_revisao', 'organizadores', 'paginas', 'autores', 'autores-endogeno','autores-endogeno-nome', 'tags', 'Hash', 'tipo_producao', 'natureza', 'titulo', 'nome_do_evento', 'ano_do_trabalho', 'pais_do_evento', 'cidade_do_evento', 'classificacao', 'periodico', 'volume', 'issn', 'estrato_qualis', 'editora', 'numero_de_paginas', 'numero_de_volumes']

    def extrair_dados(self, registro, tipo_producao):
        linha = {coluna: None for coluna in self.colunas}
        linha['tipo_producao'] = tipo_producao  # Define o tipo de produção com base na chave do dicionário
        
        # Mapear diretamente os campos do registro para a linha, assegurando que todos os campos desejados sejam extraídos
        for campo in ['titulo', 'idioma', 'periodico', 'ano', 'volume', 'issn', 'estrato_qualis', 'pais_de_publicacao', 'paginas', 'doi']:
            linha[campo] = registro.get(campo, '')

        # Tratar os autores como uma string concatenada se eles existirem
        linha['autores'] = '; '.join(registro.get('autores', []))

        # Tratar os campos 'autores-endogeno' e 'autores-endogeno-nome'
        if 'autores-endogeno' in registro and registro['autores-endogeno']:
            id_endogeno = registro['autores-endogeno'][0]
            linha['idLattes'] = id_endogeno
            linha['nome'] = registro['autores-endogeno-nome'][0].get(id_endogeno, None)
        
        # Adicionar os outros campos conforme necessário aqui
        # Por exemplo, tratamento de 'tags', 'Hash', etc., quando necessário

        return linha

    def processar_publicacoes(self):
        linhas = []
        # Itera diretamente sobre cada registro em self.data
        for registro in self.data:
            linha = self.extrair_dados(registro, registro.get('tipo_producao', 'Desconhecido'))
            linhas.append(linha)
        return linhas

    def extract_zips(self, pathzip):
        destination = os.path.join(os.getcwd(), '_data', 'in_json')
        if not os.path.exists(destination):
            os.makedirs(destination)
            print(f"Criada pasta para armazenar dados descompactados: {destination}")
        else:
            print(f"Descompactando arquivos para: {destination}")

        with zipfile.ZipFile(pathzip, 'r') as zip_ref:
            zip_ref.extractall(destination)
        
        print("Descompactação concluída...")
        return destination  # Retorna o caminho onde os arquivos foram descompactados

    def find_and_merge_publication_json_files(self, pathjson):
        all_data = []
        for filename in os.listdir(pathjson):
            if 'publication.json' in filename:
                print(f"Extraindo dados do arquivo '{filename}'...")
                with open(os.path.join(pathjson, filename), 'r', encoding='utf-8') as file:
                    file_data = json.load(file)
                    for tipo_producao in file_data:  # Para cada tipo de produção no arquivo
                        for ano in file_data[tipo_producao]:  # Para cada ano dentro de um tipo de produção
                            for registro in file_data[tipo_producao][ano]:  # Itera sobre cada registro
                                # Adicionar ou atualizar o campo 'tipo_producao' em cada registro
                                registro_atualizado = registro.copy()  # Fazer cópia para evitar modificar o original
                                registro_atualizado['tipo_producao'] = tipo_producao  # Atualizar ou adicionar o campo 'tipo_producao'
                                all_data.append(registro_atualizado)  # Adicionar o registro atualizado à lista

        # Salva a lista unificada em um novo arquivo JSON
        unified_json_path = os.path.join(pathjson, 'unified_pub.json')
        with open(unified_json_path, 'w', encoding='utf-8') as unified_file:
            json.dump(all_data, unified_file, ensure_ascii=False, indent=4)
        
        print(f"Arquivo unificado criado em: {unified_json_path}")

        # Atualiza self.data com os dados unidos
        self.data = all_data

    def merge_publication_json_files(self, pathjson):
        """
        This function receives a path to a JSON folder, accesses the folder's contents, searches for files
        containing 'publication.json' in their filename, merges their contents and saves the resulting
        data as a CSV file in destination folder.
        """
        data = []
        # Looping through the files in the given path
        for filename in os.listdir(pathjson):
            if 'publication.json' in filename:
                print(f"Extraindo dados do arquivo {filename}...")
                # Opening the file and appending its data to the list
                with open(os.path.join(pathjson, filename), 'r', encoding='utf-8') as file:
                    data.append(json.load(file))

        # Creating the output directory if it doesn't exist
        destination = os.path.join(os.getcwd(), '_data','powerbi')
        if not os.path.exists(destination):
            os.mkdir(destination)

        # Writing the merged data to a CSV file
        print(f"Criando arquivo CSV...")
        with open(os.path.join(destination, 'publication.csv'), 'w', encoding='utf-8', newline='') as csv_file:
            writer = csv.writer(csv_file)
            # Writing the header based on the keys of the first item in the list
            writer.writerow(data[0].keys())
            # Looping through the items in the list and writing them as rows in the CSV file
            for item in data:
                writer.writerow(item.values())

    def exportar_para_csv(self, nome_arquivo='publicacoes.csv'):
        linhas = self.processar_publicacoes()
        df = pd.DataFrame(linhas, columns=self.colunas)
        filepathcsv = os.path.join("./","_data","powerbi",nome_arquivo)
        df.to_csv(filepathcsv, index=False)
        print(f'Arquivo criado com sucesso em {filepathcsv}')

    def ler_csv_dados(self, pathdata,filename):
        filepath = os.path.join(pathdata,filename)
        df_pub=pd.read_csv(filepath)
        tipos = df_pub['tipo_producao'].value_counts()
        qualis=df_pub['estrato_qualis'].value_counts()  
        quantidade_nan = df_pub['estrato_qualis'].isna().sum()
        tipos_qualis = qualis.count()
        percentual_nan=np.round(quantidade_nan/tipos.sum()*100,1)
        print(f"\n{tipos.sum()} linhas no total, distribuídas nos seguintes tipos de produção:")
        print(f"{quantidade_nan} linhas ({percentual_nan}% do total de linhas) com NaN no campo estrato_qualis")
        print(tipos)
        ano_min=df_pub['ano'].min()
        ano_max=df_pub['ano'].max()
        print(f"\n{tipos.get('PERIODICO', 0)} publicações em periódicos, no período de {int(ano_min)} a {int(ano_max)}")
        percentual=np.round(qualis.sum()/tipos.get('PERIODICO', 0)*100,1)
        print(f"{qualis.sum()} publicações ({percentual}%) classificadas no qualis periodicos conforme estratificação:")
        quantidade_nan_periodico = df_pub[df_pub['tipo_producao'] == 'PERIODICO']['estrato_qualis'].isna().sum()
        percentual_nan_periodico = np.round(quantidade_nan_periodico/tipos.get('PERIODICO', 0)*100,1)
        print(f"{quantidade_nan_periodico} publicações em periódicos ({percentual_nan_periodico}%) com valor 'NaN' no estrato_qualis\n")
        print(qualis)
        return df_pub

    def ler_xls_dados(self, pathdata,filename):
        filepath = os.path.join(pathdata,filename)
        df_pub=pd.read_excel(filepath)
        tipos = df_pub['tipo_producao'].value_counts()
        qualis=df_pub['estrato_qualis'].value_counts()
        tipos_qualis = qualis.count()
        print(f"\n{tipos.sum()} linhas no total, distribuídas nos seguintes tipos de produção:")
        print(tipos)
        ano_min=df_pub['ano'].min()
        ano_max=df_pub['ano'].max()
        print(f"\n{tipos.get('PERIODICO', 0)} publicações em periódicos, no período de {int(ano_min)} a {int(ano_max)}")
        percentual=np.round(qualis.sum()/tipos.get('PERIODICO', 0)*100,1)
        print(f"{qualis.sum()} publicações ({percentual}%) classificadas no qualis periodicos conforme estratificação:")
        print(qualis)
        return df_pub

    def sigla_producao(self, tipo_producao, separador=""):
        """
        Função para converter os valores da coluna 'tipo_producao' em siglas, desconsiderando preposições,
        fazendo split de 'tipo_producao' em palavras também pelo caractere "_".
        
        Parâmetros:
        - tipo_producao: O valor da coluna 'tipo_producao'.
        - separador: O caractere para separar as iniciais. Padrão é "_".
        
        Retorna:
        - Uma string que é a sigla formada pelas iniciais das palavras em 'tipo_producao', desconsiderando preposições.
        """
        # Lista de preposições para serem ignoradas
        preposicoes = ['e', 'de', 'do', 'da', 'dos', 'das', 'em', 'na', 'no', 'nas', 'nos', 'por', 'para', 'com']

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "DTPB": 1, #DEMAIS_TIPOS_DE_PRODUCAO_BIBLIOGRAFICA 
            "TJ": 2, #TEXTO_EM_JORNAIS
            "E": 3, #EVENTO
            "CL": 4, #CAPITULO_DE_LIVRO
            "L": 5, #LIVRO
            "AA": 6, #ARTIGO_ACEITO
            "P": 7, #PERIODICO
        }

        # Fazendo split por espaço e por underscore
        palavras = tipo_producao.replace('_', ' ').replace('-', ' ').split()

        # Inicializar a lista de iniciais
        iniciais = []

        for palavra in palavras:
            # Verificar se a palavra não é uma preposição e não está vazia
            if palavra.lower() not in preposicoes and palavra:
                # Adicionar a inicial em maiúsculo à lista de iniciais
                iniciais.append(palavra[0].upper())

        # Juntar as iniciais para formar a sigla
        sigla = "".join(iniciais)

        # Aplicar o mapeamento para encontrar o número correspondente à sigla
        numero = mapeamento_siglas.get(sigla, "")

        # Retornar a string formatada com o número e a sigla
        return f"{numero}-{sigla}"  

    def sigla_qualis(self, estrato_qualis, separador=""):
        """
        Função para converter os valores da coluna 'estrato_qualis' em siglas com números de ordenação,
        
        Parâmetros:
        - estrato_qualis: O valor da coluna 'estrato_qualis'.
        - separador: O caractere para separar as iniciais. Padrão é "_".
        
        Retorna:
        - Uma string que é a sigla formada pelas iniciais das palavras em 'estrato_qualis' antecedida por seu número de ordenação.
        """

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "C": 1,
            "B4": 2,
            "B3": 3,
            "B2": 4,
            "B1": 5,
            "B2": 6,
            "B1": 7,
            "A4": 8,
            "A3": 9,
            "A2": 10,
            "A1": 11,
        }

        # Aplicar o mapeamento para encontrar o número correspondente à sigla
        numero = mapeamento_siglas.get(estrato_qualis, "")

        # Retornar a string formatada com o número e a sigla
        return f"{numero}-{estrato_qualis}"

    def agrupar_publicacoes(self, filename):
        # Caminho para o arquivo CSV
        pathfilename = os.path.join(os.getcwd(), '_data', 'powerbi', filename)
        df = pd.read_csv(pathfilename)

        # Use .loc para modificar o DataFrame diretamente e evitar o aviso
        df.loc[:, 'sigla_producao'] = df['tipo_producao'].apply(lambda x: self.sigla_producao(x))
        df_publicacoes = df[['idLattes','ano','sigla_producao']]

        # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
        contagem_publicacoes = df_publicacoes.groupby(['ano','sigla_producao','idLattes']).size().reset_index(name='contagem')

        # Retornar a contagem de orientações por tipo e por ano
        return contagem_publicacoes

    def agrupar_qualis(self, filename):
        # Caminho para o arquivo CSV
        pathfilename = os.path.join(os.getcwd(), '_data', 'powerbi', filename)
        df = pd.read_csv(pathfilename)

        # # Use .loc para modificar o DataFrame diretamente e evitar o aviso
        df.loc[:, 'estrato_qualis'] = df['estrato_qualis'].apply(lambda x: self.sigla_qualis(x))
        df_publicacoes = df[['idLattes','ano','estrato_qualis']]

        # Remover todas as linhas que contêm pelo menos um valor NaN
        df_publicacoes_limpo = df_publicacoes.dropna()

        # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
        contagem_qualis = df_publicacoes_limpo.groupby(['ano','estrato_qualis','idLattes']).size().reset_index(name='contagem')

        # Retornar a contagem de orientações por tipo e por ano
        return contagem_qualis

    def plotar_producoes_barras_empilhadas(self, filename):
        contagem_publicacoes = self.agrupar_publicacoes(filename)
        fig = make_subplots(rows=1, cols=1)
        anos = sorted(contagem_publicacoes['ano'].unique())

        mapeamento_siglas_para_numeros = {
            "1-DTPB": 1, # Demais tipos de produção bibliográfica
            "2-TJ": 2, # Texto em jornais
            "3-E": 3, # Evento
            "4-CL": 4, # Capítulo de Livro
            "5-L": 5, # Livro
            "6-AA": 6, # Artigo Aceito
            "7-P": 7, # Periódico
        }

        naturezas_ordenadas = sorted(contagem_publicacoes['sigla_producao'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

        for natureza in naturezas_ordenadas:
            contagem_por_ano = []
            labels_por_ano = [] # Para armazenar os rótulos de dados
            for ano in anos:
                contagem = contagem_publicacoes[(contagem_publicacoes['ano'] == ano) & (contagem_publicacoes['sigla_producao'] == natureza)]['contagem'].sum()
                if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4]:
                    contagem = -contagem
                contagem_por_ano.append(contagem)
                labels_por_ano.append(str(contagem)) # Convertendo a contagem em string para usar como rótulo
            
            # Adicionar a barra ao gráfico com rótulos de dados
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza, text=labels_por_ano, textposition='auto'))

        # Atualizar o layout para permitir barras empilhadas, ajustar o eixo Y para mostrar valores negativos, e garantir que todos os anos sejam mostrados no eixo X
        fig.update_layout(
            barmode='relative',
            title_text='Contagem de Produções por Tipo e Ano (Produção de divulgação plotadas abaixo do eixo X)',
            xaxis_title="Ano",
            yaxis_title="Quantidade de Produções Biliográficas",
            yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),
        )        
        # Garantir que todos os anos apareçam na barra de rótulos do eixo X
        fig.update_xaxes(tickmode='array', tickvals=anos)
        fig.show()


    def plotar_qualis_barras_empilhadas(self, filename):
        contagem_publicacoes = self.agrupar_qualis(filename)
        contagem_publicacoes.replace("-nan", np.nan, inplace=True)
        contagem_publicacoes.dropna(inplace=True)
        fig = make_subplots(rows=1, cols=1)
        anos = sorted(contagem_publicacoes['ano'].unique())

        mapeamento_siglas_para_numeros = {
            "1-C": 1, "2-B4": 2, "3-B3": 3, "4-B2": 4, "5-B1": 5,
            "6-B2": 6, "7-B1": 7, "8-A4": 8, "9-A3": 9, "10-A2": 10, "11-A1": 11,
        }
        naturezas_ordenadas = sorted(contagem_publicacoes['estrato_qualis'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))
        
        # Obter a paleta de cores 'Greens'
        cores = px.colors.sequential.Greens
        # Certificar que temos cores suficientes para todas as barras, repetindo a paleta se necessário
        num_barras = len(naturezas_ordenadas)
        cores_repetidas = cores * (num_barras // len(cores) + 1)
        
        for index, natureza in enumerate(naturezas_ordenadas):
            contagem_por_ano = []
            for ano in anos:
                contagem = contagem_publicacoes[(contagem_publicacoes['ano'] == ano) & (contagem_publicacoes['estrato_qualis'] == natureza)]['contagem'].sum()
                if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4, 5, 6, 7]:
                    contagem = -contagem
                contagem_por_ano.append(contagem)
            
            # Usar uma cor da paleta para cada barra
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza, marker_color=cores_repetidas[index]))

        fig.update_layout(
            barmode='relative',
            title_text='Contagem de Produções por Estrato Qualis e Ano (Qualis abaixo de B plotado do eixo X)',
            xaxis_title="Ano",
            yaxis_title="Quantidade de Publicações",
            yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),
        )
        # Garantir que todos os anos apareçam na barra de rótulos do eixo X
        fig.update_xaxes(tickmode='array', tickvals=anos)        
        fig.show()

class PreparadorDeOrientacoes():
    def __init__(self):
        self.data = []
        self.colunas = ['id_lattes_orientadores', 'tipo_orientacao', 'natureza', 'titulo', 'idioma', 'ano', 'id_lattes_aluno', 'nome_aluno', 'instituicao', 'pais', 'curso', 'codigo_do_curso', 'bolsa', 'agencia_financiadora', 'codigo_agencia_financiadora', 'nome_orientadores', 'tags', 'Hash']
        self.pathjson = os.path.join('_data','in_json')
        # self.filename = '863.advise.json'

        # Abrir o arquivo e carregando o JSON
    def open_file(self, filename):
        with open(os.path.join(self.pathjson, filename), 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        linhas_achatadas = self.extrair_orientacoes(data)
        df = pd.DataFrame(linhas_achatadas)
        df.to_csv('orientacoes_achatadas.csv', index=False)
        return df
    
    def agrupar_orientacoes(self, filename):
        # Caminho para o arquivo JSON
        pathfilename = os.path.join(self.pathjson, filename)

        # dict_orientacoes = pd.read_json(pathfilename)

        # Carregar o JSON e aplicar a função
        with open(pathfilename, 'r', encoding='utf-8') as file:
            data = json.load(file)

        linhas_achatadas = self.extrair_orientacoes(data)
        df = pd.DataFrame(linhas_achatadas)

        # Salvar os dados achatados em um CSV
        df.to_csv('orientacoes_achatadas.csv', index=False)

        # Use .loc para modificar o DataFrame diretamente e evitar o aviso
        df.loc[:, 'sigla_natureza'] = df['natureza'].apply(lambda x: self.sigla_natureza(x))
        df_orientacoes = df[['id_lattes_orientadores','ano','sigla_natureza']]

        # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
        contagem_orientacoes = df_orientacoes.groupby(['ano','sigla_natureza','id_lattes_orientadores']).size().reset_index(name='contagem')

        # Retornar a contagem de orientações por tipo e por ano
        return contagem_orientacoes

    # Prosseguir com a definição e uso da função para extrair as orientações
    def extrair_orientacoes(self, json_data):
        colunas = self.colunas
        linhas = []

        # Iterar sobre cada tipo de orientação e ano
        for tipo_orientacao, anos in json_data.items():
            for ano, orientacoes in anos.items():
                for orientacao in orientacoes:
                    # Preparar um dicionário para cada linha de acordo com as colunas definidas
                    linha = {}
                    # Adicionar dados específicos da orientação
                    for campo in orientacao:
                        if campo in colunas:
                            linha[campo] = orientacao[campo]
                    # Tratar 'id_lattes_orientadores' como uma lista, juntar os IDs com vírgula se houver mais de um
                    linha['id_lattes_orientadores'] = ', '.join(orientacao.get('id_lattes_orientadores', []))
                    # Adicionar o tipo de orientação como uma coluna
                    linha['tipo_orientacao'] = tipo_orientacao
                    # Adicionar o ano, garantindo que sobreponha qualquer valor de 'ano' nos dados individuais
                    linha['ano'] = ano
                    linhas.append(linha)

        return linhas

    def sigla_natureza(self, natureza, separador=""):
        """
        Função para converter os valores da coluna 'natureza' em siglas, desconsiderando preposições,
        fazendo split de 'natureza' em palavras também pelo caractere "_".
        
        Parâmetros:
        - natureza: O valor da coluna 'natureza'.
        - separador: O caractere para separar as iniciais. Padrão é "_".
        
        Retorna:
        - Uma string que é a sigla formada pelas iniciais das palavras em 'natureza', desconsiderando preposições.
        """
        # Lista de preposições para serem ignoradas
        preposicoes = ['e', 'de', 'do', 'da', 'dos', 'das', 'em', 'na', 'no', 'nas', 'nos', 'por', 'para', 'com']

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "OON": 1, #Orientações de Outra Natureza
            "IC": 2, #Iniciação Científica
            "MCCAE": 3, #Monografia de Conclusão Curso de Atualização ou Especialização
            "SPD": 4, #
            "TCCG": 5, #Trabalho de Conclusão de Curso de Graduação
            "DM": 6, #Dissertação de Mestrado
            "TD": 7, #Tese de Doutorado
        }

        # Fazendo split por espaço e por underscore
        palavras = natureza.replace('_', ' ').replace('-', ' ').split()

        # Inicializar a lista de iniciais
        iniciais = []

        for palavra in palavras:
            # Verificar se a palavra não é uma preposição e não está vazia
            if palavra.lower() not in preposicoes and palavra:
                # Adicionar a inicial em maiúsculo à lista de iniciais
                iniciais.append(palavra[0].upper())

        # Juntar as iniciais para formar a sigla
        sigla = "".join(iniciais)

        # Aplicar o mapeamento para encontrar o número correspondente à sigla
        numero = mapeamento_siglas.get(sigla, "Desconhecido")

        # Retornar a string formatada com o número e a sigla
        return f"{numero}-{sigla}"     

    def plotar_orientacoes_barras_agrupadas(self, filename):
        # montar dataframe com agrupamento de orientações por tipo por ano no programa
        contagem_orientacoes = self.agrupar_orientacoes(filename)

        # Criar uma figura com subplots
        fig = make_subplots(rows=1, cols=1)

        # Encontrar anos e naturezas únicos para as orientações
        anos = sorted(contagem_orientacoes['ano'].unique())

        # Mapeamento de exemplo conforme as siglas fornecidas
        mapeamento_siglas = {
            "OON": 1,
            "IC": 2,
            "MCCAE": 3,
            "SPD": 4,
            "TCCG": 5,
            "DM": 6,
            "TD": 7,
        }

        # Ordenar as naturezas com base nos números associados em mapeamento_siglas
        mapeamento_siglas_para_numeros = {sigla: int(sigla.split('-')[0]) for sigla in contagem_orientacoes['sigla_natureza'].unique()}
        naturezas_ordenadas = sorted(contagem_orientacoes['sigla_natureza'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

        # Criar uma barra para cada tipo de orientação em cada ano
        for natureza in naturezas_ordenadas:
            contagem_por_ano = []
            for ano in anos:
                # Somar as contagens para cada ano e natureza
                contagem = contagem_orientacoes[(contagem_orientacoes['ano'] == ano) & (contagem_orientacoes['sigla_natureza'] == natureza)]['contagem'].sum()
                contagem_por_ano.append(contagem)
            
            # Adicionar a barra ao gráfico
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza))

        # Atualizar o layout para permitir barras agrupadas
        fig.update_layout(barmode='group', title_text='Contagem de Orientações no Programa por Tipo e Ano', xaxis_title="Ano", yaxis_title="Quantidade de Orientações")

        # Mostrar o gráfico
        fig.show()


    def plotar_orientacoes_barras_empilhadas(self, filename):
        # montar dataframe com agrupamento de orientações por tipo por ano no programa
        contagem_orientacoes = self.agrupar_orientacoes(filename)
        
        # Criar uma figura com subplots
        fig = make_subplots(rows=1, cols=1)

        # Encontrar anos únicos para as orientações
        anos = sorted(contagem_orientacoes['ano'].unique())

        # Mapeamento para siglas de tipos de orientações
        mapeamento_siglas = {
            "OON": 1,
            "IC": 2,
            "MCCAE": 3,
            "SPD": 4,
            "TCCG": 5,
            "DM": 6,
            "TD": 7,
        }

        # Mapeamento para ordenar siglas por duração/complexidade
        mapeamento_siglas_para_numeros = {
            "1-OON": 1,
            "2-IC": 2,
            "3-MCCAE": 3,
            "4-SPD": 4,
            "5-TCCG": 5,
            "6-DM": 6,
            "7-TD": 7,
        }

        # Ordenar as naturezas baseando-se no número extraído da sigla
        naturezas_ordenadas = sorted(contagem_orientacoes['sigla_natureza'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

        # Criar uma barra para cada tipo de orientação em cada ano, seguindo a ordem definida
        for natureza in naturezas_ordenadas:
            contagem_por_ano = []
            for ano in anos:
                # Somar as contagens para cada ano e natureza
                contagem = contagem_orientacoes[(contagem_orientacoes['ano'] == ano) & (contagem_orientacoes['sigla_natureza'] == natureza)]['contagem'].sum()
                # Se a sigla estiver associada ao número 1 ou 2, tornar a contagem negativa
                if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4]:
                    contagem = -contagem
                contagem_por_ano.append(contagem)
            
            # Adicionar a barra ao gráfico
            fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza))

        # Atualizar o layout para permitir barras empilhadas e ajustar o eixo Y para mostrar valores negativos
        fig.update_layout(
            barmode='relative',  # Usar 'relative' para empilhar incluindo valores negativos
            title_text='Contagem de Orientações por Tipo e Ano (Orientações de curta duração plotadas abaixo do eixo X)',
            xaxis_title="Ano",
            yaxis_title="Quantidade de Orientações",
            yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),  # Destacar a linha zero
        )

        # Mostrar o gráfico
        fig.show()


### Descompactar arquivo e-lattes

In [ ]:
path = "./"
print(f"     Pasta corrente: {path}")
pathjson = os.path.join(path,'_data','in_json')
print(f"Pasta arquivos JSON: {pathjson}")
try:
    pathdata = os.path.join(path,'_data','powerbi')
    print(f" Pasta de dados CSV: {pathdata}")
except:
    print('Pasta de dados ainda não existe.')

print("\nConteúdo da pasta JSON:")
list(os.listdir(pathjson))

In [ ]:
os.listdir(pathdata)

### Arquivos na pasta de dados zipados

In [ ]:
os.listdir('./_data/in_zip')

### Lista de classificação de veículo Medicina II (Plataforma Sucupira)

In [ ]:
import os
import warnings
import pandas as pd

# Suprimir o aviso específico gerado por openpyxl
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")

# URL direto para o arquivo Excel no GitHub
url = 'https://raw.githubusercontent.com/makaires77/ppgcs/main/_data/powerbi/classificacoes_publicadas_medicina_ii_2022_1672761173777.xlsx'

## Converter .xlsx em .csv
# Ler o arquivo Excel diretamente do GitHub
df_veiculos_sucupira = pd.read_excel(url)

# Salvar o DataFrame como arquivo CSV, separado por ';'
csv_filename = 'classificacoes_publicadas_medicina_ii_2022_1672761173777.csv'
filepath = os.path.join('_data','powerbi',csv_filename)
df_veiculos_sucupira.to_csv(filepath, sep=';', index=False)

print(f'Arquivo {csv_filename} salvo com sucesso.')

In [ ]:
print(f'{len(df_veiculos_sucupira.index)} veículos classificados no arquivo da plataforma sucupira')
df_veiculos_sucupira

In [ ]:
print(f'{len(df_veiculos_sucupira.index)} veículos classificados no arquivo da plataforma sucupira')
df_veiculos_sucupira['Estrato'].value_counts().sort_index()

### Lista de classificação de veículo Medicina II (Grupo de Trabalho Cristiana)

In [ ]:
agregador = AgregadorQlattes()
pasta_dados = 'ppgcs/_data/powerbi'
arquivo_qualis_veiculos = 'MEDICINA_II1_(alterado_cristiana_2022-02-21).xlsx'
pathfilename = Path.home() / pasta_dados / arquivo_qualis_veiculos
nome_aba_veiculos = 'Veículos - Área'
df_veiculos_qualis = agregador.ler_aba_excel(pathfilename, nome_aba_veiculos)

print(f'Abas da planilha: {agregador.listar_nomes_abas(pathfilename)}')
print(f'{len(df_veiculos_qualis.index)} veículos classificados no arquivo do comitê de Medicina II')
df_veiculos_comite = df_veiculos_qualis.loc[:, ['ISSN', 'Título', 'Estrato FINAL']]
df_veiculos_comite

In [ ]:
print(f'{len(df_veiculos_comite.index)} veículos classificados no arquivo do comitê de Medicina II')
df_veiculos_comite['Estrato FINAL'].value_counts().sort_index()

In [ ]:
df_veiculos_qualis.head()

## Montar dados das publicações pelo e-Lattes

    890 é o arquivo para 55 membros do PPGCS no período 2000 a 2024, selecionada área de avaliação MedicinaII
    889 é o arquivo para 55 membros do PPGCS no período 2017 a 2024, selecionada área de avaliação MedicinaII
    863 é o arquivo para 55 membros do PPGCS no período 2000 a 2024, selecionada área de avaliação nenhuma

## Analisar extração do e-Lattes com área de avaliação: 'MedicinaII'

In [ ]:
## Descompactar arquivo zipado na pasta JSON
prep_pub = PreparadorDePublicacoes()
pathfilezip = '_data/in_zip/890.files.zip'
destination = prep_pub.extract_zips(pathfilezip)

## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Ler e montar análise exploratória
filename='publicacoes.csv'
df_pub = prep_pub.ler_csv_dados(pathdata,filename)

In [ ]:
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
print(list(df_pub.keys()))
df_pub['estrato_qualis'].value_counts(dropna=False)

In [ ]:
## Descompactar arquivo zipado na pasta JSON
prep_pub = PreparadorDePublicacoes()
pathfilezip = '_data/in_zip/863.files.zip'
destination = prep_pub.extract_zips(pathfilezip)

## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Ler e montar análise exploratória
filename='publicacoes.csv'
df_pub = prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
df_pub['estrato_qualis'].value_counts(dropna=False)

## Analisar extração do e-Lattes com área de avaliação: 'nenhuma área'

In [ ]:
## Descompactar arquivo zipado na pasta JSON:
prep_pub = PreparadorDePublicacoes()
pathfilezip = '_data/in_zip/889.files.zip'
destination = prep_pub.extract_zips(pathfilezip)

## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Ler e montar análise exploratória
filename='publicacoes.csv'
prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

## Analisando outros arquivos

    RR-2017-2023PPGCS René Rachou, 55 pesquisadores de 2017 Até 2023 extraído em 2023.06


In [ ]:
## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Ler e montar análise exploratória
filename='publicacoes.csv'
df_pub = prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
## Descompactar arquivo zipado na pasta JSON
prep_pub = PreparadorDePublicacoes()
pathfilezip = '_data/in_zip/753.files.zip'
destination = prep_pub.extract_zips(pathfilezip)

## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Ler e montar análise exploratória
filename='publicacoes.csv'
df_pub = prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
# publicacoes_v7_nenhuma_area
filename='publicacoes_v7_nenhuma_area.csv'
prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
# publicacoes_v7_medicinaII
filename = 'publicacoes_v7_medicinaII.csv'
prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
# v6
filename='202306_publicacoes_v6.csv'
prep_pub.ler_csv_dados(pathdata,filename)
prep_pub.plotar_producoes_barras_empilhadas(filename)
prep_pub.plotar_qualis_barras_empilhadas(filename)

In [ ]:
# v5
filename='publicacoes_v5.xls'
prep_pub.ler_xls_dados(pathdata,filename)
try:
    prep_pub.plotar_producoes_barras_empilhadas(filename)
    prep_pub.plotar_qualis_barras_empilhadas(filename)
except Exception as e:
    print("\nNão foi possível gerar as plotagens devido ao erro:")
    print(e)

# Montar dados das orientações

In [ ]:
print('Conteúdo da pasta dos arquivos compactados')
print(os.listdir('_data/in_zip/'))

path = "./"
print(f"     Pasta corrente: {path}")
pathjson = os.path.join(path,'_data','in_json')
print(f"Pasta arquivos JSON: {pathjson}")
try:
    pathdata = os.path.join(path,'_data','powerbi')
    print(f" Pasta de dados CSV: {pathdata}")
except:
    print('Pasta de dados ainda não existe.')

print("\nConteúdo da pasta dos arquivos JSON:")
print(list(os.listdir(pathjson)))

In [ ]:
## Descompactar arquivo zipado na pasta JSON
prep_pub = PreparadorDePublicacoes()
pathfilezip = '_data/in_zip/890.files.zip'
destination = prep_pub.extract_zips(pathfilezip)

## Unir todos aquivos de publicações (caso haja mais de um com final publication.json na pasta serão mesclados):
prep_pub.find_and_merge_publication_json_files(pathjson)

## Mapear arquivo de dados para PowerBI a partir do JSON unificado
linhas = prep_pub.processar_publicacoes()
prep_pub.exportar_para_csv()

## Gerar dados sobre as orientações
filename='890.advise.json'
# filename='889.advise.json'
pathfilename = os.path.join(pathjson,filename)
print(pathfilename)
dict_orientacoes = pd.read_json(pathfilename)
len(dict_orientacoes)

In [ ]:
prep_ort = PreparadorDeOrientacoes()
prep_ort.plotar_orientacoes_barras_agrupadas(filename)
prep_ort.plotar_orientacoes_barras_empilhadas(filename)

## Atualizar colaboração docente-discente

In [ ]:
list(os.listdir(pathdata))

### Funções tratar nomes

In [ ]:
def padronizar_titulo(titulo_bruto):
    '''Retira acentos, expressão (Org.) e espaços vazios do título da publicação
    Autor: Marcos Aires (Fev.2022)
    '''
    import unicodedata
    import re
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', titulo_bruto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.','')
    
    titulo_padronizado = string.strip().strip('"')
    
    return titulo_padronizado

def padronizar_nome(linha_texto):
    '''Procura sobrenomes e abreviaturas e monta nome completo
     Recebe: String com todos os sobrenomes e nomes, abreviados ou não
    Retorna: Nome completo no formato padronizado em SOBRENOME AGNOME, Prenomes
      Autor: Marcos Aires (Mar.2022)
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', linha_texto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.','').replace('\'','')
    string = string.replace(',,,',',').replace(',,',',')
    string = re.sub(r'[0-9]+', '', string)
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', string) if not unicodedata.combining(ch))
    
    # Expressões regulares para encontrar padrões de divisão de nomes de autores
    sobrenome_inicio   = re.compile(r'^[A-ZÀ-ú-a-z]+,')                  # Sequência de letras maiúsculas no início da string
    sobrenome_composto = re.compile(r'^[A-ZÀ-ú-a-z]+[ ][A-ZÀ-ú-a-z]+,')  # Duas sequências de letras no início da string, separadas por espaço, seguidas por vírgula
    letra_abrevponto   = re.compile(r'^[A-Z][.]')                        # Uma letra maiúscula no início da string, seguida por ponto
    letra_abrevespaco  = re.compile(r'^[A-Z][ ]')                        # Uma letra maiúscula no início da string, seguida por espaço
    letras_dobradas    = re.compile(r'[A-Z]{2}')                         # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasini = re.compile(r'[A-Z]{2}[ ]')                      # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasfim = re.compile(r'[ ][A-Z]{2}')                      # Duas letras maiúsculas juntas no final da string, precedida por espaço
    letras_duasconsnts = re.compile(r'[B-DF-HJ-NP-TV-XZ]{2}')            # Duas Letras maiúsculas e consoantes juntas
    letras_tresconsnts = re.compile(r'[B-DF-HJ-NP-TV-XZ]{3}')            # Três Letras maiúsculas e consoantes juntas
    
    # Agnomes e preprosições a tratar, agnomes vão maiúsculas para sobrenome e preposições vão para minúsculas nos nomes
    nomes=[]
    agnomes       = ['NETO','JUNIOR','FILHO','SEGUNDO','TERCEIRO','SOBRINHO']
    preposicoes   = ['de','da','do','das','dos', ' e ']
    nome_completo = ''
    
    # Ajustar lista de termos, identificar sobrenomes compostos e ajustar sobrenome com ou sem presença de vírgula
    div_sobrenome   = sobrenome_inicio.findall(string)
    div_sbrcomposto = sobrenome_composto.findall(string)
    
    # print('-'*100)
    # print('                 Recebido:',string)
    
    # Caso haja vírgulas na string, tratar sobrenomes e sobrenomes compostos
    if div_sobrenome != [] or div_sbrcomposto != []:
        # print('CASO_01: Há víruglas na string')
        div = string.split(', ')
        sobrenome     = div[0].strip().upper()
        try:
            div_espaco    = div[1].split(' ')
        except:
            div_espaco    = ['']
        primeiro      = div_espaco[0].strip('.').strip()
        
        # print('     Dividir por vírgulas:',div)
        # print('      Primeira DivVirgula:',sobrenome)
        # print('Segunda DivVrg/DivEspaços:',div_espaco)
        # print('      Primeira DivEspaços:',primeiro)
               
        # Caso primeiro nome seja somente duas letras maiúsculas juntas, trata-se de duas iniciais
        if len(primeiro)==2 or letras_tresconsnts.findall(primeiro):
            # print('CASO_01.a: Há duas letras ou três letras consoantes juntas, são iniciais')
            primeiro_nome=primeiro[0].strip()
            # print('          C01.a1_PrimNome:',primeiro_nome)
            nomes.append(primeiro[1].strip().upper())
            try:
                nomes.append(primeiro[2].strip().upper())
            except:
                pass
        else:
            # print('CASO_01.b: Primeiro nome maior que 2 caracteres')
            primeiro_nome = div_espaco[0].strip().title()
            # print('          C01.a2_PrimNome:',primeiro_nome)
        
        # Montagem da lista de nomes do meio
        for nome in div_espaco:
            # print('CASO_01.c: Para cada nome da divisão por espaços após divisão por vírgula')
            if nome not in nomes and nome.lower()!=primeiro_nome.lower() and nome.lower() not in primeiro_nome.lower() and nome!=sobrenome:   
                # print('CASO_01.c1: Se o nome não está nem como primeiro nome, nem sobrenomes')
                # print(nome, len(nome))
                
                # Avaliar se é abreviatura seguida de ponto e remover o ponto
                if len(nome)<=2 and nome.lower() not in preposicoes:
                    # print('    C01.c1.1_Nome<=02:',nome)
                    for inicial in nome:
                        # print(inicial)
                        if inicial not in nomes and inicial not in primeiro_nome:
                            nomes.append(inicial.replace('.','').strip().title())
                elif len(nome)==3 and nome.lower() not in preposicoes:
                        # print('    C01.c1.2_Nome==03:',nome)
                        for inicial in nome:
                            if inicial not in nomes and inicial not in primeiro_nome:
                                nomes.append(inicial.replace('.','').strip().title())
                else:
                    if nome not in nomes and nome!=primeiro_nome and nome!=sobrenome and nome!='':
                        if nome.lower() in preposicoes:
                            nomes.append(nome.replace('.','').strip().lower())
                        else:
                            nomes.append(nome.replace('.','').strip().title())
                        # print(nome,'|',primeiro_nome)
                        
        #caso haja sobrenome composto que não esteja nos agnomes considerar somente primeiro como sobrenome
        if div_sbrcomposto !=[] and sobrenome.split(' ')[1] not in agnomes and sobrenome.split(' ')[0].lower() not in preposicoes:
            # print('CASO_01.d: Sobrenome composto sem agnomes')
            # print(div_sbrcomposto)
            # print('Sobrenome composto:',sobrenome)
            
            nomes.append(sobrenome.split(' ')[1].title())
            sobrenome = sobrenome.split(' ')[0].upper().strip()
            # print('Sobrenome:',sobrenome)
            
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('    Nomes:',nomes)
        
        #caso haja preposição como agnome desconsiderar e passar para final dos nomes
        if div_sbrcomposto !=[] and sobrenome.split(' ')[0].lower() in preposicoes:
            # print('CASO_01.e: Preposição no Sobrenome passar para o final dos nomes')
            # print('   div_sbrcomposto:', div_sbrcomposto)
            # print('Sobrenome composto:',div_sbrcomposto)
            
            nomes.append(div_sbrcomposto[0].split(' ')[0].lower())
            # print('    Nomes:',nomes)
            sobrenome = div_sbrcomposto[0].split(' ')[1].upper().strip(',').strip()
            # print('Sobrenome:',sobrenome)
            
            for i in nomes:
                # print('CASO_01.e1: Para cada nome avaliar se o sobrenome está na lista')
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('  Nomes:',nomes)
        
        # print('Ao final do Caso 01')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('           Lista de nomes:',nomes, len(nomes),'nomes')
        
    # Caso não haja vírgulas na string considera sobrenome o último nome da string dividida com espaço vazio
    else:
        # print('CASO_02: Não há víruglas na string')
        try:
            div = string.split(' ')
            # print('      Divisões por espaço:',div)
            
            if div[-1] in agnomes: # nome final é um agnome
                sobrenome     = div[-2].upper().strip()+' '+div[-1].upper().strip()
                for i in div[1:-2]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.title().strip())
                    if i in preposicoes:
                        nomes.append(i.lower().strip())
            else:
                if len(div[-1]) > 2:
                    sobrenome     = div[-1].upper().strip()
                    primeiro_nome = div[1].title().strip()
                    for i in div[1:-1]:
                        if i != sobrenome and i not in preposicoes:
                            nomes.append(i.title().strip())
                        if i in preposicoes:
                            nomes.append(i.lower().strip())
                else:
                    sobrenome     = div[-2].upper().strip()
                    for i in div[-1]:
                        nomes.append(i.title())
                    primeiro_nome = nomes[0].title().strip()
                    for i in div[1:-1]:
                        if i != sobrenome and i not in preposicoes:
                            nomes.append(i.title().strip())
                        if i in preposicoes:
                            nomes.append(i.lower().strip())
        except:
            sobrenome = div[-1].upper().strip()
            for i in div[1:-1]:
                    if i != sobrenome and i not in preposicoes:
                        nomes.append(i.title().strip())
                    if i in preposicoes:
                        nomes.append(i.lower().strip())
            
        if sobrenome.lower() != div[0].lower().strip():
            primeiro_nome=div[0].title().strip()
        else:
            primeiro_nome=''
        
        # print('Ao final do Caso 02')
        # print('    Sobrenome sem vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome sem vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio sem vírgula:',nomes, len(nomes),'nomes')
    
    # Encontrar e tratar como abreviaturas termos com apenas uma ou duas letras iniciais juntas, com ou sem ponto
    for j in nomes:
        # print('CASO_03: Avaliar cada nome armazenado na variável nomes')
        # Procura padrões com expressões regulares na string
        div_sobrenome      = sobrenome_inicio.findall(j)
        div_sbrcomposto    = sobrenome_composto.findall(j)
        div_abrevponto     = letra_abrevponto.findall(j)
        div_abrevespaco    = letra_abrevespaco.findall(j)
        div_ltrdobradasini = letras_dobradasini.findall(j)
        div_ltrdobradasfim = letras_dobradasfim.findall(j)
        div_ltrdobradas    = letras_dobradas.findall(j)
        tamanho=len(j)
        # print('\n', div_ltrdobradasini, div_ltrdobradasfim, tamanho, 'em:',j,len(j))
        
        #caso houver abreviatura com uma letra em maiúscula nos nomes
        if div_abrevponto !=[] or tamanho==1:
            # print('CASO_03.1: Há abreviaturas uma letra maiúscula nos nomes')
            nome = j.replace('.','').strip()
            if nome not in nomes and nome != sobrenome and nome != primeiro_nome:
                # print('CASO_03.1a: Há abreviaturas uma letra maiúscula nos nomes')
                nomes.append(nome.upper())
        
        #caso houver duas inicias juntas em maiúsculas
        elif div_ltrdobradasini !=[] or div_ltrdobradasfim !=[] or div_ltrdobradas !=[] :
            # print('CASO_03.2: Há abreviaturas uma letra maiúscula nos nomes')
            for letra in j:
                # print('CASO_03.2a: Avaliar cada inicial do nome')
                if letra not in nomes and letra != sobrenome and letra != primeiro_nome:
                    # print('CASO_03.2a.1: Se não estiver adicionar inicial aos nomes')
                    nomes.append(letra.upper())
        
        #caso haja agnomes ao sobrenome
        elif sobrenome in agnomes:
            # print('CASO_03.3: Há agnomes nos sobrenomes')
            sobrenome = nomes[-1].upper()+' '+sobrenome
            # print(sobrenome.split(' '))
            # print('Sobrenome composto:',sobrenome)
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
            
        else:
            # print('CASO_03.4: Não há agnomes nos sobrenomes')
            if j not in nomes and j not in sobrenome and j != primeiro_nome:
                if len(nomes) == 1:
                    nomes.append(j.upper())
                elif 1 < len(nomes) <= 3:
                    nomes.append(j.lower())
                else:
                    nomes.append(j.title())
         
        # print('Ao final do Caso 03')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
        
    nomes_meio=' '.join([str for str in nomes]).strip()
    # print('        Qte nomes do meio:',nomes,len(nomes))
    
    if primeiro_nome.lower() == sobrenome.lower():
        # print('CASO_04: Primeiro nome é igual ao sobrenome')
        try:
            primeiro_nome=nomes_meio.split(' ')[0]
        except:
            pass
        try:
            nomes_meio.remove(sobrenome)
        except:
            pass
    
        # print('Ao final do caso 04')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
    
    # Caso sobrenome seja só de 1 letra passá-lo para nomes e considerar o próximo nome como sobrenome
    for i in range(len(div)):
        if len(sobrenome)==1 or sobrenome.lower() in preposicoes:
            # print('CASO_05: Mudar sobrenomes até o adequado')
            div    = string.split(', ')
            # print('Divisão por vírgulas:',div)
            avaliar0       = div[0].split(' ')[0].strip()
            if 1< len(avaliar0) < 3:
                # print('CASO_05.1: 1 < Sobrenome < 3 fica em minúsculas')
                sbrn0          = avaliar0.lower()
            else:
                # print('CASO_05.2: Sobrenome de tamanho 1 ou maior que 3 fica em maiúsculas')
                sbrn0          = avaliar0.title()
            # print('sbrn0:',sbrn0, len(sbrn0))
            
            try:
                avaliar1=div[0].split(' ')[1].strip()
                # print('avaliar0',avaliar0)
                # print('avaliar1',avaliar1)
                if 1 < len(avaliar1) <=3:
                    sbrn1     = avaliar1.lower()
                else:
                    sbrn1     = avaliar1.title()
                # print('sbrn1:',sbrn1, len(sbrn1))

            except:
                pass

            if div != []:
                # print('CASO_05.3: Caso haja divisão por vírgulas na string')
                try:
                    div_espaco     = div[1].split(' ')
                except:
                    div_espaco     = div[0].split(' ')
                sobrenome      = div_espaco[0].strip().upper()
                try:
                    primeiro_nome  = div_espaco[1].title().strip()
                except:
                    primeiro_nome  = div_espaco[0].title().strip()
                if len(sbrn0) == 1:
                    # print('CASO_05.3a: Avalia primeiro sobrenome de tamanho 1')
                    # print('Vai pros nomes:',str(sbrn0).title())
                    nomes_meio = nomes_meio+str(' '+sbrn0.title())
                    # print('   NomesMeio:',nomes_meio)

                elif 1 < len(sbrn0) <= 3:
                    # print('CASO_05.3b: Avalia primeiro sobrenome 1< tamanho <=3')
                    # print('Vão pros nomes sbrn0:',sbrn0, 'e sbrn1:',sbrn1)

                    div_tresconsoantes = letras_tresconsnts.findall(sobrenome)
                    if div_tresconsoantes != []:
                        # print('CASO_05.4: Três consoantes como sobrenome')
                        for letra in sobrenome:
                            nomes.append(letra)

                        if len(sobrenome) >2:
                            sobrenome=nomes[0]
                        else:
                            sobrenome=nomes[1]
                        nomes.remove(sobrenome)
                        primeiro_nome=nomes[0]
                        nomes_meio=' '.join([str for str in nomes[1:]]).strip()
                        nome_completo=sobrenome.upper()+', '+nomes_meio                
                    
                    try:                       
                        # print(' 05.3b    Lista de Nomes:',nomes_meio)
                        nomes_meio=nomes_meio.replace(sbrn0,'')
                        # print(' 05.3b ReplaceSobrenome0:',nomes_meio)
                        nomes_meio=nomes_meio.replace(sbrn1,'')
                        # print(' 05.3b ReplaceSobrenome1:',nomes_meio)
                    except Exception as e:
                        # print('   Erro ReplaceSobrenome:',e)
                        pass
                    try:
                        nomes_meio.replace(primeiro_nome.title(),'')
                        nomes_meio.replace(primeiro_nome.lower(),'')
                        nomes_meio.replace(primeiro_nome,'')
                        # print(' 05.3b Replace PrimNome:',nomes_meio)
                    except Exception as e:
                        print('Erro no try PrimeiroNome:',e)
                        pass
                    nomes_meio = nomes_meio.replace(sobrenome,'')
                    try:
                        for n,i in enumerate(avaliar1):
                            nomes.append(i.upper())
                            sbrn1     = avaliar1[0]
                        else:
                            sbrn1     = avaliar1.title()
                        # print('sbrn1:',sbrn1, len(sbrn1))
                        nomes_meio = nomes_meio+str(' '+sbrn0)+str(' '+sbrn1)
                    except:
                        nomes_meio = nomes_meio+str(' '+sbrn0)
                    nomes      = nomes_meio.strip().strip(',').split(' ')
                    # print(' 05.3b NomesMeio:',nomes_meio)
                    # print(' 05.3b     Nomes:',nome)

                else:
                    # print('CASO_05.3c: Avalia primeiro sobrenome >3')
                    nomes_meio = nomes_meio+str(' '+div[0].strip().title())
                    nomes      = nomes_meio.strip().split(' ')
                    # print(' 05.3c NomesMeio:',nomes_meio)
                    # print(' 05.3c     Nomes:',nomes)

                nomes_meio=nomes_meio.replace(sobrenome,'').replace(',','').strip()
                nomes_meio=nomes_meio.replace(primeiro_nome,'').strip()

            # print('Ao final do caso 05')
            # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
            # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
            # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
    
    if sobrenome != '' and primeiro_nome !='':
        nome_completo=sobrenome.upper().replace(',','')+', '+primeiro_nome.replace(',','')+' '+nomes_meio.replace(sobrenome,'').replace(',','')
    elif sobrenome != '':
        nome_completo=sobrenome.upper().replace(',','')+', '+nomes_meio.replace(sobrenome,'').replace(',','')
    else:
        nome_completo=sobrenome.upper()
    
#     print('Após ajustes finais')
#     print('     Sobrenome:',sobrenome)
#     print(' Primeiro Nome:',primeiro_nome)
#     print('         Nomes:',nomes)
#     print('     NomesMeio:',nomes_meio)        
        
#     print('                Resultado:',nome_completo)
    
    return nome_completo.strip()

def iniciais_nome(linha_texto):
    '''Função para retornar sobrenome+iniciais dos nomes, na forma: SOBRENOME, X Y Z
     Recebe: String com nome
    Retorna: Tupla com nome e sua versão padronizada em sobrenome+agnomes em maiúsculas, seguida de vírgula e iniciais dos nomes 
      Autor: Marcos Aires (Mar.2022)
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', linha_texto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.','')
        
    # Expressões regulares para encontrar padrões de divisão de nomes de autores
    sobrenome_inicio   = re.compile(r'^[A-ZÀ-ú-a-z]+,')                 # Sequência de letras maiúsculas no início da string
    sobrenome_composto = re.compile(r'^[A-ZÀ-ú-a-z]+[ ][A-ZÀ-ú-a-z]+,') # Duas sequências de letras no início da string, separadas por espaço, seguidas por vírgula
    letra_abrevponto   = re.compile(r'^[A-Z][.]')                       # Uma letra maiúscula no início da string, seguida por ponto
    letra_abrevespaco  = re.compile(r'^[A-Z][ ]')                       # Uma letra maiúscula no início da string, seguida por espaço
    letras_dobradas    = re.compile(r'[A-Z]{2}')                        # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasini = re.compile(r'[A-Z]{2}[ ]')                     # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasfim = re.compile(r'[ ][A-Z]{2}')                     # Duas letras maiúsculas juntas no final da string, precedida por espaço
        
    nomes=[]
    agnomes       = ['NETO','JUNIOR','FILHO','SEGUNDO','TERCEIRO', 'SOBRINHO']
    preposicoes   = ['da','de','do','das','dos','DA','DE','DOS','DAS','DOS','De']
    nome_completo = ''
    
    # Ajustar lista de termos, identificar sobrenomes compostos e ajustar sobrenome com ou sem presença de vírgula
    div_sobrenome      = sobrenome_inicio.findall(string)
    div_sbrcomposto    = sobrenome_composto.findall(string)
    
    # Caso haja vírgulas na string, tratar sobrenomes e sobrenomes compostos
    if div_sobrenome != [] or div_sbrcomposto != []:
        div   = string.split(', ')
        sobrenome     = div[0].strip().upper()
        try:
            div_espaco    = div[1].split(' ')
        except:
            div_espaco  = ['']
        primeiro      = div_espaco[0].strip('.')
        
        # Caso primeiro nome sejam somente duas letras maiúsculas juntas, trata-se de duas iniciais
        if len(primeiro)==2:
            primeiro_nome=primeiro[0].strip()
            nomes.append(primeiro[1].strip())
        else:
            primeiro_nome = div_espaco[0].strip().title()
        
        # Montagem da lista de nomes do meio
        for nome in div_espaco:
            if nome not in nomes and nome.lower()!=primeiro_nome.lower() and nome.lower() not in primeiro_nome.lower() and nome!=sobrenome:   
                # print(nome, len(nome))
                
                # Avaliar se é abreviatura seguida de ponto e remover o ponto
                if len(nome)<=2 and nome.lower() not in preposicoes:
                    for inicial in nome:
                        # print(inicial)
                        if inicial not in nomes and inicial not in primeiro_nome:
                            nomes.append(inicial.replace('.','').strip().title())
                else:
                    if nome not in nomes and nome!=primeiro_nome and nome!=sobrenome and nome!='':
                        if nome.lower() in preposicoes:
                            nomes.append(nome.replace('.','').strip().lower())
                        else:
                            nomes.append(nome.replace('.','').strip().title())
                        # print(nome,'|',primeiro_nome)
                        
        #caso haja sobrenome composto que não esteja nos agnomes considerar somente primeiro como sobrenome
        if div_sbrcomposto !=[] and sobrenome.split(' ')[1] not in agnomes:
            # print(div_sbrcomposto)
            # print('Sobrenome composto:',sobrenome)
            nomes.append(sobrenome.split(' ')[1].title())
            sobrenome = sobrenome.split(' ')[0].upper()
            # print('Sobrenome:',sobrenome.split(' '))
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
        
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
        
    # Caso não haja vírgulas na string considera sobrenome o último nome da string dividida com espaço vazio
    else:
        try:
            div       = string.split(' ')
            if div[-2] in agnomes:
                sobrenome = div[-2].upper()+' '+div[-1].strip().upper()
                for i in nomes[1:-2]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
            else:
                sobrenome = div[-1].strip().upper()
                for i in div[1:-1]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
        except:
            sobrenome = div[-1].strip().upper()
            for i in div[1:-1]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
            
        if sobrenome.lower() != div[0].strip().lower():
            primeiro_nome=div[0].strip().title()
        else:
            primeiro_nome=''
        
        # print('    Sobrenome sem vírgula:',sobrenome)
        # print('Primeiro nome sem vírgula:',primeiro_nome)
        # print('Nomes do meio sem vírgula:',nomes)
    
    # Encontrar e tratar como abreviaturas termos com apenas uma ou duas letras iniciais juntas, com ou sem ponto
    for j in nomes:
        # Procura padrões com expressões regulares na string
        div_sobrenome      = sobrenome_inicio.findall(j)
        div_sbrcomposto    = sobrenome_composto.findall(j)
        div_abrevponto     = letra_abrevponto.findall(j)
        div_abrevespaco    = letra_abrevespaco.findall(j)
        div_ltrdobradasini = letras_dobradasini.findall(j)
        div_ltrdobradasfim = letras_dobradasfim.findall(j)
        div_ltrdobradas    = letras_dobradas.findall(j)
        tamanho=len(j)
        # print('\n', div_ltrdobradasini, div_ltrdobradasfim, tamanho, 'em:',j,len(j))
        
        #caso houver abreviatura com uma letra em maiúscula nos nomes
        if div_abrevponto !=[] or tamanho==1:
            cada_nome = j.replace('.','').strip()
            if cada_nome not in nomes and cada_nome != sobrenome and nome != primeiro_nome:
                nomes.append(cada_nome)
        
        #caso houver duas inicias juntas em maiúsculas
        elif div_ltrdobradasini !=[] or div_ltrdobradasfim !=[] or div_ltrdobradas !=[] :
            for letra in j:
                if letra not in nomes and letra != sobrenome and letra != primeiro_nome:
                    nomes.append(letra)
        
        #caso haja agnomes ao sobrenome
        elif sobrenome in agnomes:
            sobrenome = nomes[-1].upper()+' '+sobrenome
            # print(sobrenome.split(' '))
            # print('Sobrenome composto:',sobrenome)
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
            
        else:
            if j not in nomes and j not in sobrenome and j != primeiro_nome:
                nomes.append(j)
    
    nomes_meio=' '.join([str[0] for str in nomes]).strip()
    # print('Qte nomes do meio',len(nomes),nomes)
    if sobrenome != '' and primeiro_nome !='':
        sobrenome_iniciais = sobrenome+', '+primeiro_nome[0]+' '+nomes_meio
    elif sobrenome != '':
        sobrenome_iniciais = sobrenome
    
    return sobrenome_iniciais.strip()


## Agregar aprendizado supervisionado humano à medida que forem sendo identificados erros na situação atual
lista_extra = [
                # ('ALBUQUERQUE, Adriano B', 'ALBUQUERQUE, Adriano Bessa'),
                # ('ALBUQUERQUE, Adriano', 'ALBUQUERQUE, Adriano Bessa'),
                # ('COELHO, Andre L V', 'COELHO, Andre Luis Vasconcelos'),
                # ('DUARTE, Joao B F', 'DUARTE, Joao Batista Furlan'),
                # ('FILHO, Raimir H','HOLANDA FILHO, Raimir'),
                # ('FILHO, Raimir','HOLANDA FILHO, Raimir'),
                # ('FORMIGO, A','FORMICO, Maria Andreia Rodrigues'),
                # ('FORMICO, A','FORMICO, Maria Andreia Rodrigues'),
                # ('FURLAN, J B D', 'FURLAN, Joao Batista Duarte'),
                # ('FURTADO, Elizabeth', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Elizabeth S', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Elizabeth Sucupira','FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, M E S', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Vasco', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, J P', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, J V P', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, Vasco', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, Elizabeth','FURTADO, Maria Elizabeth Sucupira'),
                # ('HOLANDA, Raimir', 'HOLANDA FILHO, Raimir'),
                # ('LEITE, G S', 'LEITE, Gleidson Sobreira'),
                # ('PEQUENO, T H C', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio','PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio Cavalcante', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PINHEIRO, Placido R', 'PINHEIRO, Placido Rogerio'),
                # ('PINHEIRO, Vladia', 'PINHEIRO, Vladia Celia Monteiro'),
                # ('RODRIGUES, M A F', 'RODRIGUES, Maria Andreia Formico'),
                # ('RODRIGUES, Andreia', 'RODRIGUES, Maria Andreia Formico'),
                # ('JOAO, Batista F Duarte,', 'FURLAN, Joao Batista Duarte'),
                # ('MACEDO, Antonio Roberto M de', 'MACEDO, Antonio Roberto Menescal de'),
                # ('MACEDO, D V', 'MACEDO, Daniel Valente'),
                # ('MENDONCA, Nabor C', 'MENDONCA, Nabor das Chagas'),
                # ('PEQUENO, Tarcisio', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio H', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PINHEIRO, Mirian C D', 'PINHEIRO, Miriam Caliope Dantas'),
                # ('PINHEIRO, Mirian Caliope Dantas', 'PINHEIRO, Miriam Caliope Dantas'),
                # ('PINHEIRO, P G C D', 'PINHEIRO, Pedro Gabriel Caliope Dantas'),
                # ('PINHEIRO, Pedro G C', 'PINHEIRO, Pedro Gabriel Caliope Dantas'),
                # ('PINHEIRO, Placido R', 'PINHEIRO, Placido Rogerio'),
                # ('PINHEIRO, Vladia', 'PINHEIRO, Vladia Celia Monteiro'),
                # ('ROGERIO, Placido Pinheiro', 'PINHEIRO, Placido Rogerio'),
                # ('REBOUCRAS FILHO, Pedro', 'REBOUCAS FILHO, Pedro Pedrosa'),
                # ('SAMPAIO, A', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SAMPAIO, Americo', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SAMPAIO, Americo Falcone', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SUCUPIRA, Elizabeth Furtado','FURTADO, Maria Elizabeth Sucupira'),
                ]

In [ ]:
def quebrar_partesnomes(nome):
    padrao = padronizar_nome(nome).lower()
    sobrenome  = padrao.split(',')[0].strip()
    restonomes = padrao.split(',')[1].strip().split(' ')
    try:
        partenome1 = restonomes[0].strip()
    except:
        partenome1 = ''
    try:
        partenome2 = restonomes[1].strip()
    except:
        partenome2 = ''
    try:
        partenome3 = restonomes[2].strip()
    except:
        partenome3 = ''        
    # print(f'{sobrenome:15} | {partenome1:1} | {partenome2:1} | {partenome3}')
    
    return sobrenome, partenome1, partenome2, partenome3

def quebrar_iniciais(nome):
    padrao = iniciais_nome(nome).lower()
    sobrenome  = padrao.split(',')[0].strip()
    restonomes = padrao.split(',')[1].strip().split(' ')
    try:
        partenome1 = restonomes[0].strip()
    except:
        partenome1 = ''
    try:
        partenome2 = restonomes[1].strip()
    except:
        partenome2 = ''
    try:
        partenome3 = restonomes[2].strip()
    except:
        partenome3 = ''        
    # print(f'{sobrenome:15} | {partenome1:1} | {partenome2:1} | {partenome3}')
    
    return sobrenome, partenome1, partenome2, partenome3


## compilar padrão regular expression para buscar dois termos dentro de janela de no máximo 3 palavras de distância
## em caso de dúvidas ver a fonte https://regex101.com/r/yL6dE4/1
def pesquisar_partes(sobrenome, partenome1, partenome2, partenome3):
    return re.compile(r'\b{0}(?:\W+\w+){{0,3}}\W+{1}\b|\b{0}(?:\W+\w+){{0,3}}\W+{2}\b|\b{0}(?:\W+\w+){{0,3}}\W+{3}\b'.format(sobrenome, partenome1, partenome2, partenome3), flags=re.IGNORECASE)
    # return re.compile(r'\b{0}(?:\W+\w+){{0,3}}\W+{1}\b|\b{0}(?:\W+\w+){{0,3}}\W+{2}\b|\b{0}(?:\W+\w+){{0,3}}\W+{3}\b|\b{3}(?:\W+\w+){{0,3}}\W+{0}\b|\b{2}(?:\W+\w+){{0,3}}\W+{0}\b|'.format(sobrenome, partenome1, partenome2, partenome3), flags=re.IGNORECASE)


def buscar_padrao(df_prod, sobrenome, partenome1, partenome2, partenome3):
    indices_achados=[]
    qte_achados_artigo=0
    lista_nomes=df_prod['AUTORES'].tolist()
    for n,i in enumerate(lista_nomes):
        lista_autores  = i.replace('Autores: ','').lower()
        
        ## Buscar pelos nomes de autor em cada linha de autores de artigo
        busca_autor = pesquisar_partes(sobrenome, partenome1, partenome2, partenome3)
        try:
            achar = re.search(busca_autor, lista_autores)
            if achar.span() !=None:
                print(achar.group(1))
                indices_achados.append(n)
        except:
            pass
        
    return indices_achados


In [ ]:

def ler_pastacsv(pathcsv):    
    print(pathcsv)
    import os, sys

    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if 'Artigos' in file:
            lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv

def ler_lista_docentes(pathcsv):
    try:
        l1='lista_docentes_colaboradores.csv'
        l2='lista_docentes_permanentes.csv'
        pathfile1=os.path.join(pathcsv,l1)
        pathfile2=os.path.join(pathcsv,l2)
        df_docclbr = pd.read_csv(pathfile1, header=None)
        df_docperm = pd.read_csv(pathfile2, header=None)
        lista_docentes = pd.concat([df_docperm, df_docclbr], ignore_index=True)[0].values
        print(f'{len(lista_docentes)} docentes permanentes e colaboradores encontrados')
    except Exception as e:
        print(e)
        
    return lista_docentes

def ler_lista_orientacoes(pathcsv):
    try:
        l1='lista_orientadores-discentes.csv'
        df_orientacoes = pd.read_csv(pathcsv+l1, delimiter=';', header=None)
        lista_orientadores = df_orientacoes.iloc[:,0].unique()
        lista_discentes    = df_orientacoes.iloc[:,1].unique()
        print(f'{len(lista_orientadores)} orientadores, com {len(lista_discentes)} discentes encontrados')
    except Exception as e:
        print('Erro ao gerar lista de orientações:')
        print(e)
        return df_orientacoes
        
    return lista_orientadores, lista_discentes 

In [ ]:
# Funções montagem de dataframes
def montardf_orientacoes(pathcsv):
    try:
        l1='lista_orientadores-discentes.csv'
        pathfile1 = os.path.join(pathcsv,l1)
        print(f'Buscando em orientadores/orientandos: {pathfile1}')
        df_orientacoes = pd.read_csv(pathfile1, delimiter=';', header=None)
        df_orientacoes.columns=['ORIENTADOR','DISCENTE']
        
    except Exception as e:
        print('Erro ao dividir dataframe de orientações:')
        print(e)
        return
        
    return df_orientacoes

def montardf_docentes_permanentes_colaboradores(pathcsv):
    try:
        l1='lista_docentes_colaboradores.csv'
        l2='lista_docentes_permanentes.csv'
        pathfile1 = os.path.join(pathcsv,l1)
        pathfile2 = os.path.join(pathcsv,l2)
        print(f'Buscando colaboradores em: {pathfile1}')
        print(f'Buscando   permanentes em: {pathfile2}')
        df_docclbr = pd.read_csv(os.path.join(pathcsv,l1), header=None)
        df_docperm = pd.read_csv(os.path.join(pathcsv,l2), header=None)
        df_docentes = pd.concat([df_docperm, df_docclbr], ignore_index=True)
        print(f'{len(df_docentes.index)} docentes permanentes e colaboradores encontrados')
    except Exception as e:
        print(e)
        
    return df_docentes

def montardf_producao(lista_csv):
    df_public=pd.DataFrame()
    if lista_csv:
        for nome_csv in lista_csv:
            if 'colaboradores' in nome_csv.lower():
                tipo='colaboradores'
            else:
                tipo='permanentes'
            pathfilename=os.path.join(pathcsv,nome_csv)
            df_pub = pd.read_csv(pathfilename)
            pat='\t\t\t\t\t\t\t\t\t\t\t\t \t\t\t\t\t\t\t\t\t\t\t\t\t'
            df_temp1 = df_pub.Data.str.split(pat=pat,expand=True)
            df_temp1.columns = (['TITULO','RevAut'])
            pat1='\t\t\t\t\t\t\t\t\t\t\t\t \t\t\t\t\t\t\t\t\t\t\t\t'
            df_temp2 = df_temp1.RevAut.str.split(pat=pat1,expand=True)
            df_temp2.columns = (['REVISTA','AUTORES'])
            df_temp0 = df_pub.drop(['Data'], axis=1)
            df_pub=df_temp0.merge(df_temp1['TITULO'],left_index=True,right_index=True)
            df_pub=df_pub.merge(df_temp2,left_index=True,right_index=True)
            try:
                df_pub.drop(['Issn','Natureza'], axis=1, inplace=True)
            except:
                pass
            try:
                df_pub.drop(['Tipo','Idioma'], axis=1, inplace=True)
            except:
                pass
            df_public = pd.concat([df_public, df_pub], ignore_index=True)
        print(df_public.keys())        
        ## Extrai o período com base nos dados
        inicio = min(df_public['ano'])
        final  = max(df_public['ano'])    
        total_artigos=len(df_public.index)
        print(f'Carregadas {total_artigos} publicações de artigos de docentes do programa no período de {inicio} a {final}...') 
    else:
        print('Lista de arquivos .csv com dados de Artigos vazia')
        return None
    return df_public

def similares(lista_autores, lista_grupo, limite_jarowinkler, distancia_levenshtein, lista_extra):
    """Função para aplicar padronização no nome de autor da lista de pesquisadores e buscar similaridade na lista de coautores
     Recebe: Lista de pesquisadores do grupo em análise gerada pela lista de nomes dos coautores das publicações em análise
    Utiliza: get_jaro_distance(), editdistance()
    Retorna: Lista de autores com fusão de nomes cuja similaridade esteja dentro dos limites definidos nesta função
      Autor: Marcos Aires (Fev.2022)
      
    Refazer: Inserir crítica de, mantendo sequência ordem alfabética, retornar no final nome mais extenso em caso de similaridade;
    """
    from pyjarowinkler.distance import get_jaro_distance
    from IPython.display import clear_output
    import editdistance
    import numpy as np
    import time
    
    t0=time.time()
    
    # limite_jarowinkler=0.85
    # distancia_levenshtein=6
    similares_jwl=[]
    similares_regras=[]
    similares=[]
    tempos=[]
    
    count=0
    t1=time.time()
    for i in lista_autores:
        count+=1
        if count > 0:
            tp=time.time()-t1
            tmed=tp/count*2
            tempos.append(tp)
        # print("Analisar similaridades com: ", nome_padronizado)
        
        count1=0
        for nome in lista_autores:
            if count1 > 0:
                resta=len(lista_autores)-count
                print(f'Analisando {count1:3}/{len(lista_autores)} resta analisar {resta:3} nomes. Previsão de término em {np.round(tmed*resta/60,1)} minutos')
            else:
                print(f'Analisando {count1:3}/{len(lista_autores)} resta analisar {len(lista_autores)-count1} nomes.')
            
            t2=time.time()
            count1+=1            

            try:
                similaridade_jarowinkler = get_jaro_distance(i, nome)
                print(f'{i:40} | {nome:40} | Jaro-Winkler: {np.round(similaridade_jarowinkler,2):4} Levenshtein: {editdistance.eval(i, nome)}')
                similaridade_levenshtein = editdistance.eval(i, nome)

                # inferir similaridade para nomes que estejam acima do limite ponderado definido, mas não idênticos e não muito distantes em edição
                if  similaridade_jarowinkler > limite_jarowinkler and similaridade_jarowinkler!=1 and similaridade_levenshtein < distancia_levenshtein:
                    # Crítica no nome mais extenso como destino no par (origem, destino)
                    
                    similares_jwl.append((i,nome))

            except:
                pass

            clear_output(wait=True)
    
    # Conjunto de regras de validação de similaridade
    # Monta uma lista de nomes a serem retirados antes de montar a lista de troca
    trocar=[]
    retirar=[]
    for i in similares_jwl:
        sobrenome_i = i[0].split(',')[0]
        sobrenome_j = i[1].split(',')[0]

        try:
            iniciais_i  = iniciais_nome(i[0]).split(',')[1].strip()
        except:
            iniciais_i  = ''

        try:
            iniciais_j  = iniciais_nome(i[1]).split(',')[1].strip()
        except:
            iniciais_j  = ''

        try:
            primnome_i = i[0].split(',')[1].strip().split(' ')[0].strip()
        except:
            primnome_i = ''

        try:
            primnome_j = i[1].split(',')[1].strip().split(' ')[0].strip()
        except:
            primnome_j = ''    

        try:
            inicial_i = i[0].split(',')[1].strip()[0]
        except:
            inicial_i = ''

        try:
            resto_i   = i[0].split(',')[1].strip().split(' ')[0][1:]
        except:
            resto_i   = ''

        try:
            inicial_j = i[1].split(',')[1].strip()[0]
        except:
            inicial_j = ''

        try:
            resto_j   = i[1].split(',')[1].strip().split(' ')[0][1:]
        except:
            resto_j = ''

        # Se a distância de edição entre os sobrenomes
        if editdistance.eval(sobrenome_i, sobrenome_j) > 2 or inicial_i!=inicial_j:
            retirar.append(i)
        else:
            if primnome_i!=primnome_j and len(primnome_i)>1:
                retirar.append(i)
            if primnome_i!=primnome_j and len(primnome_i)>1 and len(primnome_j)>1:
                retirar.append(i)
            if resto_i!=resto_j and resto_i!='':
                retirar.append(i)
            if len(i[1]) < len(i[0]):
                retirar.append(i)
            if len(iniciais_i) != len(iniciais_j):
                retirar.append(i)

    for i in similares_jwl:
        if i not in retirar:
            trocar.append(i)

        if iniciais_nome(i[0]) in iniciais_nome(i[1]) and len(i[0]) < len(i[1]):
            trocar.append(i)

        if iniciais_nome(i[0]) == iniciais_nome(i[1]) and len(i[0]) < len(i[1]):
             trocar.append(i)
    
    trocar=trocar+lista_extra
    trocar.sort()
    
    return trocar

def extrair_variantes(df_dadosgrupo):
    ''' Utiliza campo de Nome em Citações do currículo como filtro para obter variantes do nome de cada membro
     Recebe: Dataframe com os dados brutos do grupo de pesquisa agrupados; lista de nomes de pesquisadores de interesse
    Retorna: Lista de tuplas com pares a serem trocados da variante pelo nome padronizado na forma (origem, destino)
    '''
    filtro1   = 'Nome'
    lista_nomes = df_dadosgrupo[(df_dadosgrupo.ROTULOS == filtro1)]['CONTEUDOS'].values

    variantes=[]
    filtro='Nome em citações bibliográficas'
    variantes=df_dadosgrupo[(df_dadosgrupo.ROTULOS == filtro)]['CONTEUDOS'].to_list()

    trocar=[]
    for j in range(len(variantes)):
        padrao_destino = padronizar_nome(lista_nomes[j])
        trocar.append((lista_nomes[j], padrao_destino))
        for k in variantes[j]:
            padrao_origem = padronizar_nome(k)
            trocar.append((k, padrao_destino))
            trocar.append((padrao_origem, padrao_destino))
    
    return trocar

def tempo(start, end):
    t=end-start

    tempo = timedelta(
        weeks   = t//(3600*24*7),
        days    = t//(3600*24),
        seconds = t,
        minutes = t//(60),
        hours   = t//(3600),
        microseconds=t//1000000,
        )
    fmt='{H:2}:{M:02}:{S:02}'
    return strfdelta(tempo)

def horas(segundos): 
    return time.strftime("%H:%M:%S", time.gmtime(segundos)) 

def dias_horas_minutos(td):
    x = (td.days, td.seconds//3600, (td.seconds//60)%60, td.seconds)
    return x #(days, hrs, mins, seconds)

def strfdelta(tdelta, fmt='{H:02}h {M:02}m {S:02}s', inputtype='timedelta'):
    """Convert a datetime.timedelta object or a regular number to a custom-formatted string, 
    just like the stftime() method does for datetime.datetime objects.

    The fmt argument allows custom formatting to be specified.  Fields can 
    include seconds, minutes, hours, days, and weeks.  Each field is optional.

    Some examples:
        '{D:02}d {H:02}h {M:02}m {S:02}s' --> '05d 08h 04m 02s' (default)
        '{W}w {D}d {H}:{M:02}:{S:02}'     --> '4w 5d 8:04:02'
        '{D:2}d {H:2}:{M:02}:{S:02}'      --> ' 5d  8:04:02'
        '{H}h {S}s'                       --> '72h 800s'

    The inputtype argument allows tdelta to be a regular number instead of the  
    default, which is a datetime.timedelta object.  Valid inputtype strings: 
        's', 'seconds', 
        'm', 'minutes', 
        'h', 'hours', 
        'd', 'days', 
        'w', 'weeks'
    """

    # Convert tdelta to integer seconds.
    if inputtype == 'timedelta':
        remainder = int(tdelta.total_seconds())
    elif inputtype in ['s', 'seconds']:
        remainder = int(tdelta)
    elif inputtype in ['m', 'minutes']:
        remainder = int(tdelta)*60
    elif inputtype in ['h', 'hours']:
        remainder = int(tdelta)*3600
    elif inputtype in ['d', 'days']:
        remainder = int(tdelta)*86400
    elif inputtype in ['w', 'weeks']:
        remainder = int(tdelta)*604800

    f = Formatter()
    desired_fields = [field_tuple[1] for field_tuple in f.parse(fmt)]
    possible_fields = ('W', 'D', 'H', 'M', 'S')
    constants = {'W': 604800, 'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    values = {}
    
    for field in possible_fields:
        if field in desired_fields and field in constants:
            values[field], remainder = divmod(remainder, constants[field])
    
    return f.format(fmt, **values)


# print (timedelta(days=365, hours=8, minutes=15))
# print ("   Hoje é: " + str(date.today()))
# print ("Agora são: " + str(datetime.now()))
# print ("Um ano no futuro estaremos em:" + str(dt.today() + timedelta(days=365)))
# hoje = date.today()
# print(hoje)
# hora = dt.now()
# print(hora)
# dias_ano = date(hoje.year, 1, 1)
# if dias_ano < hoje:
#     print ("Decorridos %d dias do ano" % ((hoje - dias_ano).days))
    
# from datetime import datetime
# now= datetime.now() #get the current date and time

# #%c - local date and time, %x-local's date, %X- local's time
# print(now.strftime("%c"))
# print(now.strftime("%x"))
# print(now.strftime("%X"))

# ##### Time Formatting ####
# #%I/%H - 12/24 Hour, %M - minute, %S - second, %p - local's AM/PM
# print(now.strftime("%I:%M:%S %p")) # 12-Hour:Minute:Second:AM
# print(now.strftime("%H:%M")) # 24-Hour:Minute

### Execução

In [ ]:
pathcsv = os.path.join("_data","powerbi")
print(ler_pastacsv(pathcsv))
print(ler_lista_docentes(pathcsv))

In [ ]:
# Montar lista com arquivos .csv que tenham 'Artigo' no nome
lista_csv = ler_pastacsv(pathcsv)

# Montar dataframe com as produções bibliográficas
df_prod = montardf_producao(lista_csv)
df_prod

In [ ]:

print('Sendo:')
lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
print(f'{len(lista_titulos)} artigos distintos publicados no período')
lista_revistas = pd.Series(df_prod['REVISTA'].values).unique().tolist()
print(f'{len(lista_revistas)} revistas distintas utilizadas no período')
lista_docentes = ler_lista_docentes()
lista_orientadores, lista_discentes = ler_lista_orientacoes()

In [ ]:
df_prod[:3]

In [ ]:
sobrenome, partenome1, partenome2, partenome3 = quebrar_partesnomes('FARIA, N. R')
indices_achados=[]
qte_achados_artigo=0
for n,i in enumerate(df_prod['AUTORES'].tolist()):
    lista_autores  = i.replace('Autores: ','').lower()

    ## Buscar pelos nomes de autor em cada linha de autores de artigo
    busca_autor = pesquisar_partes(sobrenome, partenome1, partenome2, partenome3)
    achar = re.search(busca_autor, lista_autores)
    print(achar)

### Busca por discentes dentre os autores

In [ ]:
import time
t1 = time.time()

indices_discente=[]
indices_docente=[]
nomes_discentes={}
nomes_docentes={}
for m,discente in enumerate(lista_discentes):
    qte_discentes=len(lista_discentes)
    clear_output(wait=True)
    
    ## Buscar por sobrenome seguido de partes de nomes abreviados em suas iniciais
    sobrenome, partenome1, partenome2, partenome3 = quebrar_iniciais(discente)
    lista_indice_discente = buscar_padrao(sobrenome, partenome1, partenome2, partenome3)
    if lista_indice_discente != []:
        nomes_discentes[m] = (discente, lista_indice_discente)
    
    ## Buscar por sobrenome seguido partes de nomes sem abreviaturas por iniciais
    sobrenome, partenome1, partenome2, partenome3 = quebrar_partesnomes(discente)
    lista_indice_discente = buscar_padrao(sobrenome, partenome1, partenome2, partenome3)
    if lista_indice_discente != [] and lista_indice_discente not in indices_discente:
        nomes_discentes[m] = (discente, lista_indice_discente)
    
    tdec=time.time()-t1
    tres=tdec/(m+1)*(qte_discentes-m)
    print(f'Buscado discente: {discente.title():40} registro {m+1:4}/{qte_discentes:4}, buscados {m+1:4} em {horas(tdec)}, resta {qte_discentes-m}')
    
    

for m,docente in enumerate(lista_docentes):
    qte_docentes=len(lista_docentes)
    clear_output(wait=True)
    
    ## Buscar por sobrenome seguido de partes de nomes abreviados em suas iniciais
    sobrenome, partenome1, partenome2, partenome3 = quebrar_iniciais(docente)
    lista_indice_docente = buscar_padrao(sobrenome, partenome1, partenome2, partenome3)
    if lista_indice_docente != []:
        nomes_docentes[m] = (docente, lista_indice_docente)
    
    ## Buscar por sobrenome seguido partes de nomes sem abreviaturas por iniciais
    sobrenome, partenome1, partenome2, partenome3 = quebrar_partesnomes(docente)
    lista_indice_docente = buscar_padrao(sobrenome, partenome1, partenome2, partenome3)
    if lista_indice_docente != [] and lista_indice_docente not in indices_docente:
        nomes_docentes[m] = (docente, lista_indice_docente)

    tdec=time.time()-t1
    tres=tdec/(m+1)*(qte_docentes-m)
    print(f'Buscado  docente: {docente.title():40} registro {m+1:4}/{qte_docentes:4}, buscados {m+1:4} em {horas(tdec)}, resta {qte_docentes-m}')
    
t2=time.time()
tempo(t1,t2)

In [ ]:
df_participacao_discente = pd.DataFrame(nomes_discentes).T
df_participacao_discente.columns = ['DISCENTE','INDICES_ARTIGOS']
df_participacao_discente['CONTAGEM_COAUTORIAS'] = [len(x) for x in df_participacao_discente['INDICES_ARTIGOS']]

df_participacao_docente = pd.DataFrame(nomes_docentes).T
df_participacao_docente.columns = ['DOCENTE','INDICES_ARTIGOS']
df_participacao_docente['CONTAGEM_COAUTORIAS'] = [len(x) for x in df_participacao_docente['INDICES_ARTIGOS']]

In [ ]:
# for i,d in zip(df_participacao_docente['INDICES_ARTIGOS'],df_participacao_docente['DOCENTE']):
#     for n in i:
        

In [ ]:
df_participacao_docente

In [ ]:
df_participacao_discente

In [ ]:
artigos_com_docentes=[]
for m in df_participacao_docente['INDICES_ARTIGOS']:
    for n in m:
        if n not in artigos_com_docentes:
            artigos_com_docentes.append(n)
print(len(artigos_com_docentes))    
artigos_com_docentes.sort()

In [ ]:
artigos_com_discentes=[]
for m in df_participacao_discente['INDICES_ARTIGOS']:
    for n in m:
        if n not in artigos_com_discentes:
            artigos_com_discentes.append(n)
    
artigos_com_discentes.sort()
print(len(artigos_com_discentes))

In [ ]:
# lista_semparticipacaodiscente=[]
# for i in range(len(df_prod.index)):
#     if i not in lista_participacaodiscente:
#         lista_semparticipacaodiscente.append(i)

In [ ]:
# len(lista_semparticipacaodiscente)/len(df_prod.index)

In [ ]:
# len(lista_participacaodocente)/len(df_prod.index)

In [ ]:
docentes_unificada=[]
indices_docentes_unificados=[]
for i,j in zip(nomes_discentes,indices_discente):
    print(f'{i:25} | {j}')

In [ ]:
# indices_discente

In [ ]:
# indices_docente

In [ ]:
docentes_autores = pd.Series(nomes_docentes).unique()
docentes_autores.sort()
indices_com_docente = pd.Series(indices_docente).unique()
indices_com_docente.sort()
print(f'{len(docentes_autores)} docentes aparecem como autores em {len(indices_com_docente)} artigos distintos')

discente_autores = pd.Series(nomes_discentes).unique()
discente_autores.sort()
indices_com_discente = pd.Series(indices_discente).unique()
indices_com_discente.sort()
print(f'{len(discente_autores)} discentes aparecem como autores em {len(indices_com_discente)} artigos distintos')

In [ ]:
# indices_com_discente

In [ ]:
# indices_com_docente

In [ ]:
df_discente_achados = pd.DataFrame({
    'di': pd.Series(indices_discente),
    'DISCENTE': pd.Series(nomes_discentes),
})
df_discente_achados = df_discente_achados.sort_values('di')
df_discente_achados.reset_index(inplace=True, drop=True)
df_artigos_discentes = df_discente_achados.groupby(['di'], as_index=False, sort=False).agg('; '.join)

df_docente_achados = pd.DataFrame({
    'do': pd.Series(indices_docente),
    'DOCENTE': pd.Series(nomes_docentes),
})
df_docente_achados = df_docente_achados.sort_values('do')
df_docente_achados.reset_index(inplace=True, drop=True)
df_artigos_docentes = df_docente_achados.groupby(['do'], as_index=False, sort=False).agg('; '.join)

In [ ]:
df_producao_docentes = pd.merge(df_prod, df_artigos_docentes,  left_index=True, right_on='do')
df_producao_docentes

In [ ]:
lista_artigos_docentes = []
coautorias_docentes=[]
for i,j in zip(indices_docente, nomes_docentes):
    # print(f'{i:3} {j}')
    if i not in lista_artigos_docentes:
        lista_artigos_docentes.append(j)
    else:
        pass
        # lista_artigos_docentes.extend(j)

In [ ]:
# import pandas library
import pandas as pd
  
# dictionary with dictionary object in values i.e. nested dictionary
details = { 
    0 : {
        'Name' : 'Ankit',
        'Age' : 22,
        'University' : 'BHU'
        },
    1 : {
        'Name' : 'Aishwarya',
        'Age' : 21,
        'University' : 'JNU'
        },
    2 : {
        'Name' : 'Shaurya',
        'Age' : 23,
        'University' : 'DU'
        }
}
  
# creating a Dataframe object from nested dictionary
# in which inside dictionary key is act as index value
# and column value is 0, 1, 2...
df = pd.DataFrame(details)
  
# swap the columns with indexes
df = df.transpose()
  
df

In [ ]:
# df_docentes = montardf_docentes_permanentes_colaboradores()

In [ ]:
# df_orientacoes = montardf_orientacoes()

In [ ]:
# import os
# import tkinter as tk
# from tkinter import filedialog
# from file_manipulation import FileManipulation

# root = tk.Tk()

# # Pergunta ao usuário se ele deseja criar uma nova pasta ou escolher uma já existente
# selection = input("Digite (N) para criar uma nova pasta ou tecle enter para escolher uma existente")

# # Define a pasta onde serão salvas as análises
# if selection.lower() == "n":
#     folder_path = filedialog.askdirectory(initialdir="./", title="Selecione uma pasta")
# else:
#     folder_path = "./analises"

#     # Caso a pasta já exista, avisa o usuário que os arquivos antigos serão substituídos
#     if len(os.listdir(folder_path)) > 0:
#         confirmation = input("\nAtenção: a pasta selecionada já contém arquivos. Eles serão substituídos. Deseja continuar (S/N)? ")
#         if confirmation.lower() != "s":
#             root.destroy()
#             exit()

# fm = FileManipulation(master=root)